In [182]:
import pandas as pd
import numpy as np
import requests
import os
import sys
import time
import lyricsgenius
import re
import traceback    
from ratelimit import limits, sleep_and_retry
import time
import psycopg2
from psycopg2 import sql
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from pprint import pprint
import logging
import datetime

In [184]:
def get_pg_conn():
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        dbname='spotify_demo_db',
        user='',## masked here - I have the code in the script files.
        password='',## masked here - I have the code in the script files.
        host='localhost',
        port='5432')
    return conn

In [185]:
def drop_all_tables(database_name, conn):
    try:
        cursor = conn.cursor()
        # Get a list of all tables in the current schema
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
        tables = cursor.fetchall()

        # Drop each table
        for table in tables:
            table_name = table[0]
            drop_table_query = sql.SQL("DROP TABLE IF EXISTS {} CASCADE").format(sql.Identifier(table_name))
            cursor.execute(drop_table_query)
            print(f"Dropped table: {table_name}")

        # Commit changes and close the connection
        conn.commit()
        print("All tables dropped successfully.")
    except Exception as e:
        print(f"Error: {e}")

conn = get_pg_conn()

In [192]:
def connect_apis():
    # Define your Spotify API credentials
    spotify_client_id = credentials['spotify_client_id']
    spotify_client_secret = credentials['spotify_client_secret']
    spotify_redirect_uri = credentials['spotify_redirect_uri']  # This should match your Spotify App's redirect URI
    
    # Define your Genius API credentials
    genius_client_access_token = credentials['genius_client_access_token']
    
    # Create a SpotifyOAuth object to authenticate and get access token
    sp_oauth = SpotifyOAuth(client_id=spotify_client_id, client_secret=spotify_client_secret, redirect_uri=spotify_redirect_uri, 
                            scope="user-library-read user-read-recently-played user-read-private user-read-email user-top-read user-read-playback-state user-modify-playback-state")
    
    # Create a Spotipy object with the authenticated SpotifyOAuth object
    sp = spotipy.Spotify(auth_manager=sp_oauth)
    
    genius = lyricsgenius.Genius(genius_client_access_token)
    return sp, genius

sp, genius = connect_apis()

In [190]:
user_id_df = pd.read_csv("user_id.csv")
user_id_list = user_id_df["user_id"]
user_id_list

0        hpuzeducv3xo1a831qgw84vap
1        ygk025r29ofb0yexgz9jkfxdo
2                      filtrisland
3                     sanchibhaley
4     313v3mo6bvh5cihtfmz6pvkm3rhy
5                       1221129264
6        hf9ry3zgffalheg7ucrayg9tz
7     31b6xt4erwdqhdzkne5ct57nzfm4
8                    alltrapnation
9                          spotify
10    313jpb7nsygrjtu7owt42lsug3dm
11                      1261873101
12                 monstercatmedia
13       21ehubo72mwp25kyaf7ap5ufi
14    31c2l4urfvgrs337zjhekdmfbrom
15                liquicityrecords
16       d141n7rke9bt3048fq2cspljq
17       qcgfvdv2oezedi2req0mzc389
Name: user_id, dtype: object

In [191]:
## find_user details for each user
def get_users_details(sp, user_ids):
    # List to store user details
    users_details = []

    count_insertions = 0

    for user_id in user_ids:
        try:
            conn = get_pg_conn()
            cursor = conn.cursor()

            # Check if the user already exists in the table
            cursor.execute("""
                SELECT 1 FROM user_details WHERE user_id = %s
            """, (user_id,))
            existing_user = cursor.fetchone()

            if existing_user:
                pprint(f"User with ID {user_id} already exists. Skipping insertion.")
            else:
                # Get details for the specified user
                user_info = sp.user(user_id)

                # Append user details to the list
                user_details = {
                    "user_id": user_info['id'],
                    "user_name": user_info['display_name'],
                    "user_followers": user_info['followers']['total'],
                    "user_url": user_info['external_urls']['spotify'],
                    "user_added_on": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")                
                }
                users_details.append(user_details)

                # Insert user details into PostgreSQL table
                cursor.execute("""
                    INSERT INTO user_details (user_id, user_name, user_followers, user_url, user_added_on)
                    VALUES (%s, %s, %s, %s, %s)
                """, (user_details["user_id"], 
                      user_details["user_name"], 
                      user_details["user_followers"], 
                      user_details["user_url"], 
                      user_details["user_added_on"]))
                conn.commit()
                pprint(f"User {user_id} added successfully.")
                count_insertions += 1
        except spotipy.SpotifyException as e:
            print(f"Error for user {user_id}: {e}")
            traceback.print_exc()

        finally:
            cursor.close()
            conn.close()
    pprint(f"Added {count_insertions} users.")
    return users_details

user_details = get_users_details(sp, user_id_list)

'User with ID hpuzeducv3xo1a831qgw84vap already exists. Skipping insertion.'
'User with ID ygk025r29ofb0yexgz9jkfxdo already exists. Skipping insertion.'
'User with ID filtrisland already exists. Skipping insertion.'
'User with ID sanchibhaley already exists. Skipping insertion.'
'User with ID 313v3mo6bvh5cihtfmz6pvkm3rhy already exists. Skipping insertion.'
'User with ID 1221129264 already exists. Skipping insertion.'
'User with ID hf9ry3zgffalheg7ucrayg9tz already exists. Skipping insertion.'
'User with ID 31b6xt4erwdqhdzkne5ct57nzfm4 already exists. Skipping insertion.'
'User with ID alltrapnation already exists. Skipping insertion.'
'User with ID spotify already exists. Skipping insertion.'
'User with ID 313jpb7nsygrjtu7owt42lsug3dm already exists. Skipping insertion.'
'User with ID 1261873101 already exists. Skipping insertion.'
'User with ID monstercatmedia already exists. Skipping insertion.'
'User with ID 21ehubo72mwp25kyaf7ap5ufi already exists. Skipping insertion.'
'User with

In [ ]:
# user_id_list = user_id_list[:1]
# user_id_list

In [8]:
## Find all user playlists
def create_playlist_entry(playlist_id, playlist_name, playlist_description, playlist_url, playlist_updated_on):
    """
    Create or update an entry in the 'playlist' table.
    """
    conn = get_pg_conn()
    cursor = conn.cursor()
    insert_query = """
        INSERT INTO playlist (playlist_id, playlist_name, playlist_description, playlist_url, playlist_updated_on)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT (playlist_id) DO UPDATE
        SET playlist_name = CASE WHEN playlist.playlist_name <> EXCLUDED.playlist_name THEN EXCLUDED.playlist_name ELSE playlist.playlist_name END,
            playlist_description = CASE WHEN playlist.playlist_description <> EXCLUDED.playlist_description THEN EXCLUDED.playlist_description ELSE playlist.playlist_description END,
            playlist_url = CASE WHEN playlist.playlist_url <> EXCLUDED.playlist_url THEN EXCLUDED.playlist_url ELSE playlist.playlist_url END,
            playlist_updated_on = EXCLUDED.playlist_updated_on
        WHERE playlist.playlist_id = EXCLUDED.playlist_id
    """

    cursor.execute(insert_query, (playlist_id, playlist_name, playlist_description, playlist_url, playlist_updated_on))
    conn.commit()

def create_user_playlist_entry(user_id, playlist_id):
    """
    Create an entry in the 'user_playlists' table.
    """
    conn = get_pg_conn()
    cursor = conn.cursor()
    insert_query = """
        INSERT INTO user_playlist (user_id, playlist_id)
        VALUES (%s, %s)
        ON CONFLICT DO NOTHING
    """

    cursor.execute(insert_query, (user_id, playlist_id))
    conn.commit()

def process_user_playlists(user_id):
    """
    Process user playlists and add entries to the 'playlist' and 'user_playlists' tables.
    """
    playlists = sp.user_playlists(user_id)
    total_songs = 0
    for playlist in playlists['items']:
        playlist_id = playlist['id']
        playlist_name = playlist['name']
        playlist_description = playlist['description']
        playlist_url = playlist['external_urls']['spotify']
        playlist_updated_on = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        create_playlist_entry(playlist_id, playlist_name, playlist_description, playlist_url, playlist_updated_on)
        create_user_playlist_entry(user_id, playlist_id)
        playlist_tracks = sp.playlist_tracks(playlist_id)
        total_songs += playlist_tracks['total']
    return total_songs
        

sp, genius = connect_apis()
for user_id in user_id_list:
    print(user_id, process_user_playlists(user_id))

hpuzeducv3xo1a831qgw84vap 944
ygk025r29ofb0yexgz9jkfxdo 2213
filtrisland 1604
sanchibhaley 1235
313v3mo6bvh5cihtfmz6pvkm3rhy 599
1221129264 4588
hf9ry3zgffalheg7ucrayg9tz 12333
31b6xt4erwdqhdzkne5ct57nzfm4 554
alltrapnation 1977
spotify 4434
313jpb7nsygrjtu7owt42lsug3dm 10
1261873101 2038
monstercatmedia 8302
21ehubo72mwp25kyaf7ap5ufi 1310
31c2l4urfvgrs337zjhekdmfbrom 369
liquicityrecords 7672
d141n7rke9bt3048fq2cspljq 2250
qcgfvdv2oezedi2req0mzc389 624


In [9]:
def fetch_result_set(query):
    try:
        conn = get_pg_conn()
        
        cursor = conn.cursor()
        cursor.execute(query)

        # Fetch all track IDs
        track_ids = cursor.fetchall()

        # Close the cursor and connection
        cursor.close()
        conn.close()

        return track_ids

    except Exception as e:
        print(f"Error: {e}")
        return None

In [10]:
# Find track details:
def create_track_entry(track):
    """
    Create an entry in the 'track' table.
    """
    try:
        conn = get_pg_conn()
        cur = conn.cursor()
        insert_query = """
            INSERT INTO track (
                track_id, track_album, track_available_markets,
                track_disc_number, track_explicit, track_href, track_is_local,
                track_name, track_preview_url, track_number, track_uri, track_added_on
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (track_id) DO NOTHING
        """
    
        cur.execute(insert_query, (
            track['id'], track['album']['name'],
            ', '.join(track['available_markets']), track['disc_number'], track['explicit'],
            track['href'], track['is_local'], track['name'], track['preview_url'],
            track['track_number'], track['uri'], datetime.datetime.now()
        ))
        conn.commit()
    except:
        traceback.print_exc()
        print('insert failed')
    return track['id']

def create_artist_entry(artist):
    """
    Create an entry in the 'artist' table.
    """
    try:
        conn = get_pg_conn()
        cur = conn.cursor()
        print(artist)
        sys.exit()
        # followers = artist['followers']['total'] if 'followers' in artist else None
        # genres = ', '.join(artist['genres']) if 'genres' in artist else None
        # popularity = artist['popularity'] if 'popularity' in artist else None
    
        # insert_query = """
        #     INSERT INTO artist (artist_id, artist_name, followers, genres, popularity, artist_added)
        #     VALUES (%s, %s, %s, %s, %s, %s)
        #     ON CONFLICT (artist_id) DO NOTHING
        #     RETURNING artist_id
        # """
    
        insert_query = """
            INSERT INTO artist (artist_id, artist_name, followers, artist_added)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (artist_id) DO UPDATE
            SET followers = EXCLUDED.followers
        """
    
        cur.execute(insert_query, (
            artist['id'], artist['name'], artist['followers']['total'], datetime.datetime.now()
        ))
        # cur.execute(insert_query, (
        #     artist['id'], artist['name'], followers, genres, popularity, datetime.datetime.now()
        # ))
        conn.commit()
    except:
            traceback.print_exc()
            print('insert failed')
    return artist['id']

def create_track_artist_entry(track_id, artist_id):
    """
    Create an entry in the 'track_artist' table.
    """
    try:
        conn = get_pg_conn()
        cur = conn.cursor()
        insert_query = """
            INSERT INTO track_artist (track_id, artist_id)
            VALUES (%s, %s)
            ON CONFLICT DO NOTHING
        """
    
        cur.execute(insert_query, (track_id, artist_id))
        conn.commit()
        
    except:
        traceback.print_exc()
        print('insert failed')

def fetch_tracks_from_playlist(playlist_id):
    """
    Fetch all tracks from a playlist and add them to the 'track', 'artist', and 'track_artist' tables with pagination.
    """
    offset = 0
    limit = 50  # Adjust the limit based on your needs
    print(playlist_id, "initiated.")
    while True:
        results = sp.playlist_tracks(playlist_id, offset=offset, limit=limit)

        if not results['items']:
            break

        for item in results['items']:
            track = item['track']
            if track and track['id']:
                track_id = create_track_entry(track)
    
                for artist in track['artists']:
                    artist_id = create_artist_entry({'id': artist['id'], 'name': artist['name']})
                    create_track_artist_entry(track_id, artist_id)

        offset += limit
    print(playlist_id, "completed.")
    print("--")

query = "SELECT DISTINCT playlist_id FROM playlist"
playlists = [playlist[0] for playlist in fetch_result_set(query)]
playlists = playlists[1:2]
playlists

['52Qk0Vp5fNZCeqjohcXx3a']

In [23]:
def update_artist_followers():
    try:
        conn = get_pg_conn()
        cur = conn.cursor()

        # Step 1: Fetch all distinct artist_ids
        cur.execute("SELECT DISTINCT artist_id FROM artist")
        artist_ids = [row[0] for row in cur.fetchall()]

        # Step 2: Use Spotify API to get updated information for multiple artists
        for i in range(0, len(artist_ids), 50):
            batch_artist_ids = artist_ids[i:i+50]
            artists_info = sp.artists(batch_artist_ids)['artists']

            # Step 3: Update the 'artist' table
            for artist_info in artists_info:
                artist_id = artist_info['id']
                followers = artist_info['followers']['total']
                artist_name = artist_info['name']
                artist_popularity = artist_info['popularity']

                cur.execute("""
                    UPDATE artist
                    SET followers = %s, artist_name = %s, popularity = %s
                    WHERE artist_id = %s
                """, (followers, artist_name, artist_popularity, artist_id))
            time.sleep(5)

        conn.commit()
    except:
        traceback.print_exc()
        print('Update failed')
    finally:
        conn.close()

In [24]:
def update_audio_features():
    try:
        conn = get_pg_conn()
        cur = conn.cursor()

        # Step 1: Fetch all distinct track_ids
        cur.execute("SELECT DISTINCT track_id FROM track")
        track_ids = [row[0] for row in cur.fetchall()]

        # Step 2: Use Spotify API to get updated information for multiple tracks
        for i in range(0, len(track_ids), 50):
            batch_track_ids = track_ids[i:i+50]
            tracks_info = sp.tracks(batch_track_ids)['tracks']

            # Step 3: Insert into the 'track_details' table
            for track_info in tracks_info:
                track_id = track_info['id']
                track_popularity = track_info['popularity']
                track_duration_ms = track_info['duration_ms']

                cur.execute("""
                    INSERT INTO audio_features (track_id, track_popularity, track_duration_ms)
                    VALUES (%s, %s, %s)
                    ON CONFLICT (track_id) DO NOTHING
                """, (track_id, track_popularity, track_duration_ms))
            time.sleep(10)

        conn.commit()
    except:
        traceback.print_exc()
        print('Update failed')
    finally:
        conn.close()

In [197]:
def get_recently_played():
    try:
        current_date = datetime.datetime.now().strftime('%Y-%m-%d')

        # Calculate the start time for today
        start_time_today = datetime.datetime.strptime(f'{current_date} 13:00:00', '%Y-%m-%d %H:%M:%S')
        
        # Convert the start time to milliseconds
        start_time_ms = int(start_time_today.timestamp()) * 1000
        
        # Get recently played tracks from today onward
        results = sp.current_user_recently_played(after=start_time_ms)
        
        # Extract track information
        recently_played_tracks = results['items']
        
        # Print or process the track information as needed
        for track in recently_played_tracks:
            track_name = track['track']['name']
            artist_name = track['track']['artists'][0]['name']
            print(f"Track: {track_name}, Artist: {artist_name}")

        return recently_played_tracks

    except spotipy.SpotifyException as e:
        print(f"Error: {e}")
        return None

get_recently_played()

Track: Transformations - Keeno Remix, Artist: Maduk
Track: Nasini El Donya, Artist: Ragheb Alama
Track: Upside Down (feat. Poppy Baskcomb), Artist: MEDUZA
Track: Doo Da Dub - Jon Void Remix, Artist: T & Sugah
Track: Transformations - Keeno Remix, Artist: Maduk
Track: Let Me Out - BCee Remix, Artist: T & Sugah


[{'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6TaaqqCMRMSpvNHClfnbEL'},
      'href': 'https://api.spotify.com/v1/artists/6TaaqqCMRMSpvNHClfnbEL',
      'id': '6TaaqqCMRMSpvNHClfnbEL',
      'name': 'Maduk',
      'type': 'artist',
      'uri': 'spotify:artist:6TaaqqCMRMSpvNHClfnbEL'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',
     'SK',
     'ES',
     'SE',
     'CH',
     'TW',
     'TR',
     'UY',
     'US',
     'GB',
     'AD

In [204]:
# Extract and insert relevant information into the database
for item in results['items']:
    track_id = item['track']['id']
    
    track_name = item['track']['name']
    cur.execute("SELECT COUNT(*) FROM track WHERE track_id = %s", (track_id,))
    track_exists = cur.fetchone()[0]
    print(track_id, track_name, track_exists)

3keUgTGEoZJt0QkzTB6kHg Truffle Butter 0
6PGoSes0D9eUDeeAafB2As LOVE. FEAT. ZACARI. 0
1EaKU4dMbesXXd3BrLCtYG ELEMENT. 0
6HZILIRieu8S0iqY8kIKhj DNA. 0
7KXjTSCq5nL1LoYtL7XAwS HUMBLE. 0
6SwRhMLwNqEi6alNPVG00n LOYALTY. FEAT. RIHANNA. 0
3iVcZ5G6tvkXZkZKlMpIUs Alright 0
5pUL8veHvMV8q8wFYSZnji Transformations - Keeno Remix 1
1qQqHzG5JAYD2bV1DGf5v7 Nasini El Donya 1
2UDycsru4dEa1UFmyIvKYU Upside Down (feat. Poppy Baskcomb) 1
6NizbDwlTD2e7Gx3faLMwJ Doo Da Dub - Jon Void Remix 1
2XVy03UoZ25qF2a9nlgixJ Transformations - Keeno Remix 1
3M6lcJQaCmc6Oi1czb4YIP Let Me Out - BCee Remix 1
0W6I02J9xcqK8MtSeosEXb Get Into It (Yuh) 0
4HDC7Mo6uLTujRvHymiXDf Mwaki 0
1IHWl5LamUGEuP4ozKQSXZ Tití Me Preguntó 0
3NWRapFZSlpHXO38LW21rw La leçon particulière 0
6t4VHEcJ7Kek9pgGHlDp6R Push To Start (feat. No/Me) 0
3k45oaugo4fin6M1KrtO8f Bongoloco 0
7dDLAQ1y0qOq3E4DhNh8FS Talamanca 0
4nFrcGM7MY1mpoQCC7Kefj Me & U 0
1MVqeIAwhD4T44AKVkIfic leavemealone 0
1GL7vwDnPqCnJ2xGNxNLwG These Nights 0
1ACFweuuvf6MHtptObgreR Giving

In [210]:
pip install better_profanity

     ---------------------------------------- 0.0/46.1 kB ? eta -:--:--
     ---------------------------------------- 46.1/46.1 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [207]:
results = sp.current_user_recently_played()
for item in results['items']:
    print(item['track']['name'], item['track']['id'])

Go Off 22yUER14ZEhGkhu4dFcNrW
Radar 2d0lv7SP23N5kFp9Ukis37
With You 0c7YR1XbbWcIUznxLsIov5
Doo Da Dub - Jon Void Remix 6NizbDwlTD2e7Gx3faLMwJ
Running 3ziNTpKUDUCYnqreuV5ti2
Running - Whiney Remix 49lhOe89EqXkUFerqUFdGw
Truffle Butter 3keUgTGEoZJt0QkzTB6kHg
LOVE. FEAT. ZACARI. 6PGoSes0D9eUDeeAafB2As
ELEMENT. 1EaKU4dMbesXXd3BrLCtYG
DNA. 6HZILIRieu8S0iqY8kIKhj
HUMBLE. 7KXjTSCq5nL1LoYtL7XAwS
LOYALTY. FEAT. RIHANNA. 6SwRhMLwNqEi6alNPVG00n
Alright 3iVcZ5G6tvkXZkZKlMpIUs
Transformations - Keeno Remix 5pUL8veHvMV8q8wFYSZnji
Nasini El Donya 1qQqHzG5JAYD2bV1DGf5v7
Upside Down (feat. Poppy Baskcomb) 2UDycsru4dEa1UFmyIvKYU
Doo Da Dub - Jon Void Remix 6NizbDwlTD2e7Gx3faLMwJ
Transformations - Keeno Remix 2XVy03UoZ25qF2a9nlgixJ
Let Me Out - BCee Remix 3M6lcJQaCmc6Oi1czb4YIP
Get Into It (Yuh) 0W6I02J9xcqK8MtSeosEXb
Mwaki 4HDC7Mo6uLTujRvHymiXDf
Tití Me Preguntó 1IHWl5LamUGEuP4ozKQSXZ
La leçon particulière 3NWRapFZSlpHXO38LW21rw
Push To Start (feat. No/Me) 6t4VHEcJ7Kek9pgGHlDp6R
Bongoloco 3k45oaugo4fin6

In [208]:
def update_user_activity():
    try:
        conn = get_pg_conn()
        cur = conn.cursor()
        
        current_date = datetime.datetime.now().strftime('%Y-%m-%d')
        
        # Calculate the start time for today
        start_time_today = datetime.datetime.strptime(f'{current_date} 13:00:00', '%Y-%m-%d %H:%M:%S')
        
        # Convert the start time to milliseconds
        start_time_ms = int(start_time_today.timestamp()) * 1000
        
        # Get recently played tracks from today onward
        results = sp.current_user_recently_played(after=start_time_ms)
        # Get the user's recently played tracks
        results = sp.current_user_recently_played(limit=50)
        # List to store tracks that don't exist in the 'track' table
        missing_tracks = []

        # Extract and insert relevant information into the database
        for item in results['items']:
            try:
                track_id = item['track']['id']
                played_at = item['played_at']
    
                # Check if the track exists in the 'track' table
                cur.execute("SELECT COUNT(*) FROM track WHERE track_id = %s", (track_id,))
                track_exists = cur.fetchone()[0]
                cur.execute("""
                        INSERT INTO user_activity (user_id, track_id, played_at)
                        VALUES (%s, %s, %s)
                    """, (user_id, track_id, played_at))
                print("Inserted", item['track']['name'])
            except:
                print("Could not insert", item['track']['name'])
        conn.commit()

        return missing_tracks

    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()
    finally:
        conn.close()

# Iterate over user IDs and get user activity for each user
for user_id in user_ids[:1]:
    needed_tracks = update_user_activity()

Inserted Go Off
Inserted Radar
Inserted With You
Inserted Doo Da Dub - Jon Void Remix
Inserted Running
Inserted Running - Whiney Remix
Could not insert Truffle Butter
Could not insert LOVE. FEAT. ZACARI.
Could not insert ELEMENT.
Could not insert DNA.
Could not insert HUMBLE.
Could not insert LOYALTY. FEAT. RIHANNA.
Could not insert Alright
Could not insert Transformations - Keeno Remix
Could not insert Nasini El Donya
Could not insert Upside Down (feat. Poppy Baskcomb)
Could not insert Doo Da Dub - Jon Void Remix
Could not insert Transformations - Keeno Remix
Could not insert Let Me Out - BCee Remix
Could not insert Get Into It (Yuh)
Could not insert Mwaki
Could not insert Tití Me Preguntó
Could not insert La leçon particulière
Could not insert Push To Start (feat. No/Me)
Could not insert Bongoloco
Could not insert Talamanca
Could not insert Me & U
Could not insert leavemealone
Could not insert These Nights
Could not insert Giving Me
Could not insert Giving Me
Could not insert Push To

In [132]:
def play_tracks(track_ids, duration=60):
    try:
        # Start playback for each track ID
        for track_id in track_ids:
            print("Playing ", track_id)
            sp.start_playback(uris=[f'spotify:track:{track_id}'], device_id="1bde90a0241afa4921d89fe2a0e9c29b593a72d4")
            
            
            # Sleep for the specified duration
            time.sleep(duration)

            # Pause the playback
            sp.pause_playback()

            # Wait for a moment before playing the next track
            time.sleep(2)

    except Exception as e:
        print(f"Error: {e}")


In [137]:
conn = get_pg_conn()
cur = conn.cursor()

# Fetch all user IDs from the 'user' table
cur.execute("SELECT track_id FROM track")
track_ids = [row[0] for row in cur.fetchall()]

track_ids = track_ids[20:]
print(track_ids)

['3lLOuE6Z5jlQVqO2bbpC8A', '2UDycsru4dEa1UFmyIvKYU', '2k9kWgFLCiAeRBTmwlGbAv', '3ziNTpKUDUCYnqreuV5ti2', '6NizbDwlTD2e7Gx3faLMwJ', '0c7YR1XbbWcIUznxLsIov5', '2d0lv7SP23N5kFp9Ukis37', '22yUER14ZEhGkhu4dFcNrW', '6v9qm1Y0ZGYHlVVGjbdeUh', '0cC8F5Vc4d3fsrUPc2Tdob', '6c1pzxSDUAjuzoTBI5y9ol', '6XZVVYHrnYYmiv70IlBYTG', '4tJ5ZiVcub4muuKvwubVYf', '0BA0FHgWNvFCeDff6fLu7q', '4BWm9gWCP3ZUBWTgLN0FcS', '0av697tc282yeLXrEbnamz', '0JGyRDykYSbwV0Pcx22APN', '5ZhM5Rvwsm7DNJhpsMkuSt', '2aTuHt6TPwZkT0079MzPHV', '3tjgImXDQ8IOdITzqqKVrr', '4WvBgvjsQVqSeaVu6DkFi7', '5AKvQCaELGM2uJW3F8Y0Pu', '65ZLr3sqK3XgdBIhvaZO30', '3M6lcJQaCmc6Oi1czb4YIP', '3Acp9rPxjxpoVN7prcRGq9', '6xqrKnGSZh8NnKJmmTzxvD', '6ZFLwESYdvEj0QvScyQFBK', '3PDznDdamJQVcizepENvJF', '16ZY99SwEcLkCPaKMdTQO7', '5kFvxN3ttvwt3YRd203Jt2', '03m2zQHNlsF9wJ6pUIBYft', '27BzC5OKsozvj1NWRRjObk', '3VJ7HIvH1f3sVjGRLeX0Oo', '0X8oWAC2f4GWz5NMOVq4BQ', '5tN4CvNZgVhhvCBRXAAktA', '6GHCZvaqWvAJ59ZPOWxlbs', '0MR5pH0K4SJje9EtFnKiYS', '2hdsHRqUXQItzcjcUStpTb', '1eJbLRxsIH

In [139]:
#play_tracks(track_ids)

Playing  3lLOuE6Z5jlQVqO2bbpC8A
Playing  2UDycsru4dEa1UFmyIvKYU
Playing  2k9kWgFLCiAeRBTmwlGbAv
Playing  3ziNTpKUDUCYnqreuV5ti2
Playing  6NizbDwlTD2e7Gx3faLMwJ
Playing  0c7YR1XbbWcIUznxLsIov5
Playing  2d0lv7SP23N5kFp9Ukis37
Playing  22yUER14ZEhGkhu4dFcNrW
Playing  6v9qm1Y0ZGYHlVVGjbdeUh
Playing  0cC8F5Vc4d3fsrUPc2Tdob
Playing  6c1pzxSDUAjuzoTBI5y9ol
Playing  6XZVVYHrnYYmiv70IlBYTG
Playing  4tJ5ZiVcub4muuKvwubVYf
Playing  0BA0FHgWNvFCeDff6fLu7q
Playing  4BWm9gWCP3ZUBWTgLN0FcS
Playing  0av697tc282yeLXrEbnamz
Playing  0JGyRDykYSbwV0Pcx22APN
Playing  5ZhM5Rvwsm7DNJhpsMkuSt
Playing  2aTuHt6TPwZkT0079MzPHV
Playing  3tjgImXDQ8IOdITzqqKVrr
Playing  4WvBgvjsQVqSeaVu6DkFi7
Playing  5AKvQCaELGM2uJW3F8Y0Pu
Playing  65ZLr3sqK3XgdBIhvaZO30
Playing  3M6lcJQaCmc6Oi1czb4YIP
Playing  3Acp9rPxjxpoVN7prcRGq9
Playing  6xqrKnGSZh8NnKJmmTzxvD
Playing  6ZFLwESYdvEj0QvScyQFBK
Playing  3PDznDdamJQVcizepENvJF
Playing  16ZY99SwEcLkCPaKMdTQO7
Playing  5kFvxN3ttvwt3YRd203Jt2
Playing  03m2zQHNlsF9wJ6pUIBYft
Playing 

In [118]:
def get_device_id():
    try:
        # Get the user's available devices
        devices = sp.devices()

        # Print the available devices and their IDs
        for device in devices['devices']:
            print(f"Device Name: {device['name']}, Device ID: {device['id']}")

        # Return the first device ID as an example
        if devices['devices']:
            return devices['devices'][0]['id']
        else:
            print("No devices found.")
            return None

    except Exception as e:
        print(f"Error: {e}")

# Call the function to get the user's device ID
device_id = get_device_id()

device_id

Device Name: iPhone, Device ID: 1bde90a0241afa4921d89fe2a0e9c29b593a72d4
Device Name: Web Player (Chrome), Device ID: 4c05757badd8b844d544a67a7f7b2159e19dbaec
Device Name: Web Player (Chrome), Device ID: a74cbda0c5f18cfb63ad43b354f1b6993764ed44
Device Name: Sree's Echo Dot, Device ID: ff702c65-5e7f-495f-b688-f39dd9a77844_amzn_1


'1bde90a0241afa4921d89fe2a0e9c29b593a72d4'

In [140]:
sp, genius = connect_apis()
playlist_count = 0
for playlist_id in playlists:
    playlist_count += 1
    if playlist_count >= 0:
        print(playlist_count)
        fetch_tracks_from_playlist(playlist_id)

1
52Qk0Vp5fNZCeqjohcXx3a initiated.
{'id': '6jsS2mOTAxVrlSUWiPLXpH', 'name': 'T & Sugah'}
insert failed
{'id': '4YTBo7qadslqj8V8FMRuqK', 'name': 'Whiney'}
insert failed
{'id': '25bz6UkMqlOoaXOk5UIeSW', 'name': 'Maddy'}
insert failed


Traceback (most recent call last):
  File "C:\Users\sreek\AppData\Local\Temp\ipykernel_62652\3971811826.py", line 39, in create_artist_entry
    sys.exit()
SystemExit
Traceback (most recent call last):
  File "C:\Users\sreek\AppData\Local\Temp\ipykernel_62652\3971811826.py", line 39, in create_artist_entry
    sys.exit()
SystemExit
Traceback (most recent call last):
  File "C:\Users\sreek\AppData\Local\Temp\ipykernel_62652\3971811826.py", line 39, in create_artist_entry
    sys.exit()
SystemExit


{'id': '5DNApAt05XowaylsOJo1eW', 'name': 'Voicians'}
insert failed
{'id': '6jsS2mOTAxVrlSUWiPLXpH', 'name': 'T & Sugah'}
insert failed
{'id': '1kuxUeEwGS2oBGcvP3ofbp', 'name': 'Jon Void'}
insert failed


Traceback (most recent call last):
  File "C:\Users\sreek\AppData\Local\Temp\ipykernel_62652\3971811826.py", line 39, in create_artist_entry
    sys.exit()
SystemExit
Traceback (most recent call last):
  File "C:\Users\sreek\AppData\Local\Temp\ipykernel_62652\3971811826.py", line 39, in create_artist_entry
    sys.exit()
SystemExit
Traceback (most recent call last):
  File "C:\Users\sreek\AppData\Local\Temp\ipykernel_62652\3971811826.py", line 39, in create_artist_entry
    sys.exit()
SystemExit


{'id': '7miXLG9boDOGHJaEelSL7T', 'name': 'Andromedik'}
insert failed
{'id': '5kVFEq0ZL9wPGx6nnJDcm8', 'name': 'Dualistic'}
insert failed
{'id': '5BGIoAtF2cwgcydpnUyFkF', 'name': 'Polygon'}
insert failed


...

{'id': '4xQ2BGOBUXgjxO2PAhrIyS', 'name': 'Ayah Marar'}
insert failed
{'id': '3kTwX6b6epyenw7IqEIGoT', 'name': 'OIO'}
insert failed


Traceback (most recent call last):
  File "C:\Users\sreek\AppData\Local\Temp\ipykernel_62652\3971811826.py", line 39, in create_artist_entry
    sys.exit()
SystemExit
Traceback (most recent call last):
  File "C:\Users\sreek\AppData\Local\Temp\ipykernel_62652\3971811826.py", line 39, in create_artist_entry
    sys.exit()
SystemExit


{'id': '26PnCYGyS31j6bvJjaK6J9', 'name': 'Dan Dakota'}
insert failed


Traceback (most recent call last):
  File "C:\Users\sreek\AppData\Local\Temp\ipykernel_62652\3971811826.py", line 39, in create_artist_entry
    sys.exit()
SystemExit


52Qk0Vp5fNZCeqjohcXx3a completed.
--


In [60]:
def get_song_lyrics(song_name, artist_name):
    try:
        # Get the first track found on Spotify
        sp, genius = connect_apis()

        # Search for the song on Genius using the artist and title
        genius_results = genius.search_song(song_name, artist_name)
        if genius_results:
            # Print and return the lyrics
            print(f"Lyrics for '{song_name}' by {artist_name}:\n")
            #print(genius_results.lyrics)
            return genius_results.lyrics
        else:
            print(f"Lyrics for '{song_name}' not found on Genius.")
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

In [32]:
sp, lyrics = connect_apis()

In [62]:
## find all tracks in the user
def create_track_entry(track):
    """
    Create an entry in the 'track' table.
    """
    try:
        conn = get_pg_conn()
        cur = conn.cursor()
        insert_query = """
            INSERT INTO track (
                track_id, track_album, track_artists, track_available_markets,
                track_disc_number, track_explicit, track_href,
                track_is_local, track_name, track_preview_url,
                track_number, track_uri, track_added_on
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (track_id) DO NOTHING
        """
    
        cur.execute(insert_query, (
            track['id'], track['album']['name'], None,
            ', '.join(track['available_markets']), track['disc_number'],
            track['explicit'], track['href'], track['is_local'], track['name'], track['preview_url'], track['track_number'],
            track['uri'], datetime.datetime.now()
        ))
        conn.commit()
        return track['id']
    except:
        traceback.print_exc()
        print('insert failed')

def create_audio_features_entry(track):
    """
    Create an entry in the 'track' table.
    """
    try:
        conn = get_pg_conn()
        cur = conn.cursor()
        insert_query = """
            INSERT INTO audio_features (
                track_id, track_duration_ms, track_popularity
            )
            VALUES (%s, %s, %s)
            ON CONFLICT (track_id) DO NOTHING
        """
    
        cur.execute(insert_query, (track['id'], track['duration_ms'], track['popularity']))
        conn.commit()
    except:
        traceback.print_exc()
        print('insert failed')

def create_playlist_track_entry(playlist_id, track_id):
    """
    Create an entry in the 'playlist_track' table.
    """
    try:
        conn = get_pg_conn()
        cur = conn.cursor()
        insert_query = """
            INSERT INTO playlist_track (playlist_id, track_id)
            VALUES (%s, %s)
            ON CONFLICT DO NOTHING
        """
    
        cur.execute(insert_query, (playlist_id, track_id))
        conn.commit()
    except:
        traceback.print_exc()
        print('insert failed')

def create_lyrics_entry(track_id, lyrics):
    """
    Create an entry in the 'lyrics' table.
    """
    try:
        conn = get_pg_conn()
        cur = conn.cursor()
        insert_query = """
            INSERT INTO lyrics (track_id, track_lyrics)
            VALUES (%s, %s)
            ON CONFLICT (track_id) DO NOTHING
        """
    
        cur.execute(insert_query, (track_id, lyrics))
        conn.commit()
    except:
        traceback.print_exc()
        print('insert failed')

def create_genre_entry(genre_name):
    try:
        conn = get_pg_conn()
        cursor = conn.cursor()
        # Replace 'your_genre_table' with your actual genre table name and 'your_genre_column' with the column where the genre is stored
        insert_query = sql.SQL("""
            INSERT INTO genre (genre_name)
            VALUES (%s)
            ON CONFLICT (genre_name) DO NOTHING
        """)
        cursor.execute(insert_query, (genre_name,))
        conn.commit()
    except Exception as e:
        print(f"Error creating genre entry: {e}")

def create_artist_genre(artist_id, genre_name):
    try:
        # Replace 'your_artist_genre_table', 'your_artist_id_column', and 'your_genre_id_column' with your actual table and column names
        conn = get_pg_conn()
        cursor = conn.cursor()
        
        insert_query = sql.SQL("""
            INSERT INTO artist_genre (artist_id, genre_name)
            VALUES (%s, %s)
            ON CONFLICT DO NOTHING
        """)
        cursor.execute(insert_query, (artist_id, genre_name))
        conn.commit()
    except Exception as e:
        print(f"Error creating artist-genre entry: {e}")

def get_lyrics(track_title, artist_name):
    """
    Get lyrics for a track using the Genius API.
    """
    try:
        song = genius.search_song(track_title, artist_name)
        if song:
            return song.lyrics
    except Exception as e:
        print(f"Error fetching lyrics: {e}")
    return None

def process_playlist_tracks(playlist_id):
    """
    Process tracks in a playlist and add entries to the 'track', 'playlist_track', and 'lyrics' tables.
    """
    offset = 0
    limit = 50  # You can adjust this value based on your needs

    while True:
        results = sp.playlist_tracks(playlist_id, offset=offset, limit=limit)
        if not results['items']:
            break

        for item in results['items']:
            track = item['track']
            if track and track['id']:
                track_id = create_track_entry(track)
                create_audio_features_entry(track)
                create_playlist_track_entry(playlist_id, track['id'])
    
                # Fetch lyrics and store in the 'lyrics' table
                lyrics = get_lyrics(track['name'], track['artists'][0]['name'])
                create_lyrics_entry(track['id'], lyrics)
                
        offset += limit

def fetch_result_set(query):
    try:
        conn = get_pg_conn()
        
        cursor = conn.cursor()
        cursor.execute(query)

        # Fetch all track IDs
        track_ids = cursor.fetchall()

        # Close the cursor and connection
        cursor.close()
        conn.close()

        return track_ids

    except Exception as e:
        print(f"Error: {e}")
        return None


playlist_count = 0
for playlist_id in playlists:
    playlist_count += 1
    if playlist_count >= 0:
        sp, genius = connect_apis()
        print(playlist_count)
        process_playlist_tracks(playlist_id)

1
Searching for "Running - Whiney Remix" by T & Sugah...
Done.
Searching for "Running" by Voicians...
Done.
Searching for "Doo Da Dub - Jon Void Remix" by T & Sugah...
Done.
Searching for "With You" by Andromedik...
Done.
Searching for "Radar" by Dualistic...
Done.
Searching for "Go Off" by Maduk...
Done.
Searching for "Days Away - Aktive Remix" by T & Sugah...
Done.
Searching for "Pool Hopping" by Polaris...
Done.
Searching for "All Starts To Fall" by Cardinal Sound...
Done.
Searching for "Ocean" by Keeno...
Specified song does not contain lyrics. Rejecting.
Searching for "Gravity" by Rameses B...
Done.
Searching for "Worlds Collide" by hayve...
Done.
Searching for "What I Want" by Artino...
Done.
Searching for "Kinetic" by Makoto...
Specified song does not contain lyrics. Rejecting.
Searching for "Euphoria" by T & Sugah...
Done.
Searching for "LFG" by Jon Void...
Done.
Searching for "Feel Alive" by Voicians...
Done.
Searching for "Melting Hearts" by Edlan...
Specified song does not c

In [ ]:
def create_artist_entry(artist_id, artist_name, followers, genres, popularity):
    """
    Create an entry in the 'artist' table.
    """
    conn = get_pg_conn()
    cur = conn.cursor()
    insert_query = """
        INSERT INTO artist (artist_id, artist_name, followers, genres, popularity, artist_added)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (artist_id) DO NOTHING
    """

    cur.execute(insert_query, (artist_id, artist_name, followers, genres, popularity, datetime.now()))
    conn.commit()

def create_album_entry(album_id, album_name, album_type, album_total_tracks, album_artists, album_href, album_uri, album_release_date):
    """
    Create an entry in the 'album' table.
    """
    conn = get_pg_conn()
    cur = conn.cursor()
    insert_query = """
        INSERT INTO album (album_id, album_name, album_type, album_total_tracks, album_artists, album_href, album_uri, album_release_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (album_id) DO NOTHING
    """

    cur.execute(insert_query, (album_id, album_name, album_type, album_total_tracks, album_artists, album_href, album_uri, album_release_date))
    conn.commit()

def process_playlist_tracks(playlist_id):
    """
    Process tracks in a playlist and add entries to the 'track', 'playlist_track', 'artist', and 'album' tables.
    """
    offset = 0
    limit = 50  # Adjust the limit based on your needs

    while True:
        results = sp.user_playlist_tracks(user_id, playlist_id, offset=offset, limit=limit)

        if not results['items']:
            break

        for item in results['items']:
            track = item['track']

            # Create entries for artist and album if not already present
            for artist in track['artists']:
                create_artist_entry(artist['id'], artist['name'], None, None, None)  # You may fetch more details about the artist from Spotify API

            for artist in track['artists']:
                create_artist_entry(artist['id'], artist['name'], None, None, None)  # You may fetch more details about the artist from Spotify API

            create_album_entry(
                track['album']['id'], track['album']['name'], track['album']['album_type'],
                track['album']['total_tracks'], ', '.join([artist['name'] for artist in track['album']['artists']]),
                track['album']['href'], track['album']['uri'], track['album']['release_date']
            )

        offset += limit

query = "SELECT DISTINCT playlist_id FROM playlist"
playlists = [playlist[0] for playlist in fetch_result_set(query)]

for playlist_id in playlists:
    process_playlist_tracks(playlist_id)

In [6]:
## find following for each tracks:
    ## artist
    ## album
    ## lyrics
    ## audio_features

In [64]:
sp, lyrics = connect_apis()
sp.user("weebee")

{'display_name': 'weebee',
 'external_urls': {'spotify': 'https://open.spotify.com/user/weebee'},
 'href': 'https://api.spotify.com/v1/users/weebee',
 'id': 'weebee',
 'images': [],
 'type': 'user',
 'uri': 'spotify:user:weebee',
 'followers': {'href': None, 'total': 0}}

In [1]:
def create_album_track_entry(album_id, track_id):
    """
    Create an entry in the 'album_track' table.
    """
    conn = get_pg_conn()
    cur = conn.cursor()
    insert_query = """
        INSERT INTO album_track (album_id, track_id)
        VALUES (%s, %s)
        ON CONFLICT DO NOTHING
    """

    cur.execute(insert_query, (album_id, track_id))
    conn.commit()

def create_album_entry(track):
    """
    Create an entry in the 'track_artist' table.
    """

    try:
        conn = get_pg_conn()
        cur = conn.cursor()
        insert_query = """
            INSERT INTO album (album_id, album_name, album_type, album_total_tracks, album_href, album_uri, album_release_date)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT DO NOTHING
        """
    
        cur.execute(insert_query, (track['album']['id'], track['album']['name'], track['album']['album_type'],
                    track['album']['total_tracks'],
                    track['album']['href'], track['album']['uri'], track['album']['release_date']))
        conn.commit()
        return True
    except:
        traceback.print_exc()
        return False

def create_album_artist_entry(album_id, artist_id):
    """
    Create an entry in the 'track_artist' table.
    """
    conn = get_pg_conn()
    cur = conn.cursor()
    insert_query = """
        INSERT INTO album_artist (album_id, artist_id)
        VALUES (%s, %s)
        ON CONFLICT DO NOTHING
    """

    cur.execute(insert_query, (album_id, artist_id))
    conn.commit()

def process_playlist_albums(playlist_id, all_artists):
    """
    Process tracks in a playlist and add entries to the 'track', 'playlist_track', 'artist', 'album', 'album_track', and 'track_artist' tables.
    """
    offset = 0
    limit = 50  # Adjust the limit based on your needs

    failures = 0
    while True:
        results = sp.user_playlist_tracks(user_id, playlist_id, offset=offset, limit=limit)

        if not results['items']:
            break

        for item in results['items']:
            track = item['track']

            # # Create entries for artist and album if not already present
            # for artist in track['artists']:
            #     artist_id = artist['id']
            #     create_artist_entry(artist_id, artist['name'], None, None, None)  # Basic entry without detailed info

            #     # Fetch detailed info about the artist from Spotify
            #     followers, genres, popularity = fetch_artist_details(artist_id)

            #     # Update the entry with detailed info
            #     create_artist_entry(artist_id, artist['name'], followers, genres, popularity)

            #     # Create entry in the track_artist bridge table
            #     create_track_artist_entry(track['id'], artist_id)


            completed = create_album_entry(track)

            if completed:
                for artist in track['album']['artists']:
                    if artist['id'] in all_artists:
                        create_album_artist_entry(track['album']['id'], artist['id'])
                    else:
                        failures += 1
    
                # Create entry in the album_track bridge table
                create_album_track_entry(track['album']['id'], track['id'])

        offset += limit
    return failures


In [43]:
def get_user_playlists(sp, user_id):
    try:
        # Initialize an empty list to store all playlists
        all_playlists = []

        # Set the initial offset to 0
        offset = 0

        while True:
            # Get playlists with the current offset
            playlists = sp.user_playlists(user_id, offset=offset)

            # If there are no more playlists, break out of the loop
            if not playlists['items']:
                break
            # Extract relevant information from the playlist objects and append to the list
            all_playlists.extend([{'id': playlist['id'], 'name': playlist['name'], 'href': playlist['href'], 'description': playlist['description']} 
                                  for playlist in playlists['items']])

            # Increment the offset for the next request
            offset += len(playlists['items'])
        return all_playlists

    except Exception as e:
        print(f"Error: {e}")
        return None

user_id = "spotify"
user_playlists = get_user_playlists(sp, user_id)

playlists = dict()

# Print or process the retrieved playlists
if user_playlists:
    print("User Playlists:")
    for playlist in user_playlists:
        playlists[playlist['id']] = dict()
        playlists[playlist['id']]['id'] = playlist['id']
        playlists[playlist['id']]['name'] = playlist['name']
        playlists[playlist['id']]['description'] = playlist['description']
        playlists[playlist['id']]['href'] = playlist['href']
        print(f"Playlist ID: {playlist['id']}, Playlist Name: {playlist['name']}")
else:
    print("Failed to retrieve user playlists.")

print(len(playlists.keys()))

Error: 'SpotifyOAuth' object has no attribute 'user_playlists'
Failed to retrieve user playlists.
0


In [50]:
from pprint import pprint
import psycopg2

# Add your get_pg_conn() function for database connection
def get_audio_features_for_large_set(track_ids, sp):
    try:
        # Initialize an empty list to store all audio features
        all_audio_features = []

        # Split the track IDs into chunks of 100 (maximum allowed per request)
        track_id_chunks = [track_ids[i:i+100] for i in range(0, len(track_ids), 100)]

        # Fetch audio features for each chunk
        for chunk in track_id_chunks:
            audio_features_chunk = sp.audio_features(chunk)
            all_audio_features.extend(audio_features_chunk)

        return all_audio_features

    except Exception as e:
        print(f"Error fetching audio features: {e}")
        return None

def fill_audio_features_table(audio_features, sp):

    if audio_features:

        # Insert audio features into the 'audio_features' table
        for idx, features in enumerate(audio_features, start=1):
            try:
                # Connect to the PostgreSQL database
                conn = get_pg_conn()
                cursor = conn.cursor()

                if features['id']:
                    cursor.execute("""
                        INSERT INTO audio_features (
                            id, danceability, energy, key, loudness, mode,
                            speechiness, acousticness, instrumentalness, liveness,
                            valence, tempo, duration_ms, time_signature
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", 
                        (features['id'], features['danceability'], features['energy'],
                        features['key'], features['loudness'], features['mode'],
                        features['speechiness'], features['acousticness'],
                        features['instrumentalness'], features['liveness'],
                        features['valence'], features['tempo'],
                        features['duration_ms'], features['time_signature']))
                    
                    conn.commit()
                    print(f"Inserted record {idx} of {len(audio_features)}")
                    
                
            except Exception as e:
                print(f"Error inserting record {idx}: {e}")

            finally:
                # Close the cursor and connection
                cursor.close()
                conn.close()

        print(f"Successfully inserted {len(audio_features)} records into the 'audio_features' table.")
    else:
        print("Failed to retrieve audio features.")

def fetch_result_set(query):
    try:
        conn = get_pg_conn()
        
        cursor = conn.cursor()
        cursor.execute(query)

        # Fetch all track IDs
        track_ids = cursor.fetchall()

        # Close the cursor and connection
        cursor.close()
        conn.close()

        return track_ids

    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
query = "SELECT track_id FROM track"
fetched_track_ids = fetch_result_set(query)
track_ids = [tracks[0] for tracks in fetched_track_ids]

In [94]:
def drop_all_tables(database_name, conn):
    try:
        cursor = conn.cursor()
        # Get a list of all tables in the current schema
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
        tables = cursor.fetchall()

        # Drop each table
        for table in tables:
            table_name = table[0]
            drop_table_query = sql.SQL("DROP TABLE IF EXISTS {} CASCADE").format(sql.Identifier(table_name))
            cursor.execute(drop_table_query)
            print(f"Dropped table: {table_name}")

        # Commit changes and close the connection
        conn.commit()
        print("All tables dropped successfully.")
    except Exception as e:
        print(f"Error: {e}")

conn = get_pg_conn()
drop_all_tables("spotify_staging_db", conn)

Dropped table: audio_features
Dropped table: playlist
Dropped table: artist
Dropped table: lyrics
Dropped table: track
Dropped table: user_details
Dropped table: playlist_track
Dropped table: user_playlist
Dropped table: album
All tables dropped successfully.


In [ ]:
conn = get_pg_conn()
cursor = conn.cursor()

# Create Playlist Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS playlist (
        playlist_id VARCHAR(255) PRIMARY KEY,
        playlist_name VARCHAR(255),
        playlist_description TEXT,
        playlist_url VARCHAR(255)
    )
''')

# # Create Track Table
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS tracks (
#         track_id VARCHAR(255) PRIMARY KEY,
#         name VARCHAR(255),
#         artist VARCHAR(255),
#         album VARCHAR(255),
#         duration_ms INT,
#         popularity INT
#     )
# ''')

# # Create PlaylistTrack Table
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS playlist_track (
#         playlist_id VARCHAR(255) REFERENCES playlists(playlist_id),
#         track_id VARCHAR(255) REFERENCES tracks(track_id),
#         PRIMARY KEY (playlist_id, track_id)
#     )
# ''')

# # Create User Table
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS users (
#         user_id VARCHAR(255) PRIMARY KEY,
#         display_name VARCHAR(255),
#         email VARCHAR(255),
#         country VARCHAR(255)
#     )
# ''')

# Commit changes and close the connection
conn.commit()

In [ ]:
conn = get_pg_conn()
cursor = conn.cursor()

for playlist_id, playlist in playlists.items():
    print(playlist)
    cursor.execute('''INSERT INTO playlist (playlist_id, playlist_name, playlist_description, playlist_url)
                    VALUES (%s, %s, %s, %s)''', (playlist['id'], playlist['name'], playlist['description'], playlist['href']))
conn.commit()

In [89]:
def get_playlist_tracks(sp, playlist_id):
    try:
        # Get playlist tracks using the playlist_tracks endpoint
        playlist_tracks = sp.playlist_tracks(playlist_id)
        pprint(playlist_tracks)
    except Exception as e:
        print(f"Error: {e}")
        return None

sp.playlist_tracks("37i9dQZF1DXcBWIGoYBM5M")

{'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXcBWIGoYBM5M/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2023-11-24T05:00:00Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
    'href': 'https://api.spotify.com/v1/users/',
    'id': '',
    'type': 'user',
    'uri': 'spotify:user:'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2LIk90788K0zvyj2JJVwkJ'},
       'href': 'https://api.spotify.com/v1/artists/2LIk90788K0zvyj2JJVwkJ',
       'id': '2LIk90788K0zvyj2JJVwkJ',
       'name': 'Jack Harlow',
       'type': 'artist',
       'uri': 'spotify:artist:2LIk90788K0zvyj2JJVwkJ'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE'

In [90]:
def get_playlist_tracks(sp, playlist_id):
    try:
        # Get playlist tracks using the playlist_tracks endpoint
        playlist_tracks = sp.playlist_tracks(playlist_id)

        # Extract relevant information from the track objects
        tracks_data = []

        for track in playlist_tracks['items']:
            track_info = track['track']
            tracks_data.append({
                'id': track_info['id'],
                'name': track_info['name'],
                'artist': [artist['name'] for artist in track['track']['artists']],  # Assuming the first artist for simplicity
                'album': track_info['album']['name'],
                'duration_ms': track_info['duration_ms'],
                'popularity': track_info['popularity'],
                'preview_url': track_info['preview_url'],
                'track_number': track_info['track_number'],
                'uri': track_info['uri'],
                'is_local': track_info['is_local'],
                'href': track_info['href'],
                'explicit': track_info['explicit'],
                'disc_number': track_info['disc_number'],
            })

        return tracks_data

    except Exception as e:
        print(f"Error: {e}")
        return None

In [423]:
conn = get_pg_conn()
cursor = conn.cursor()

# Create Track Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS track (      
        track_id VARCHAR(255) PRIMARY KEY,
        track_album VARCHAR(255),
        track_artists VARCHAR(255),
        track_available_markets TEXT,
        track_disc_number INTEGER,
        track_duration_ms DECIMAL,
        track_explicit BOOLEAN,
        track_href VARCHAR(255),
        track_is_local BOOLEAN,
        track_name VARCHAR(255),
        track_popularity DECIMAL,
        track_preview_url VARCHAR(255),
        track_number INTEGER,
        track_uri VARCHAR(255)
    )
''')


# track_external_ids TEXT[],
# track_external_urls TEXT[],

# Commit changes and close the connection
conn.commit()

In [134]:
def change_quotes(input_value):
    if isinstance(input_value, str):
        return input_value.replace("'", '"')
    elif isinstance(input_value, int):
        return str(input_value)
    else:
        return input_value

In [4]:
def get_tracks(playlist_id):
    try:
        # Get playlist tracks using the playlist_tracks endpoint
        playlist_tracks = sp.playlist_tracks(playlist_id)

        # Extract relevant information from the track objects
        tracks_data = []

        for track in playlist_tracks['items']:
            try:
                track_info = track['track']
                tracks_data.append({
                    'track_id': track_info['id'],
                    'track_album': track_info['album']['name'],
                    'track_artists': f"{', '.join([artist['name'] for artist in track_info['artists']])}",
                    'track_available_markets': f"[{', '.join([available_markets for available_markets in track_info['available_markets']])}]",
                    'track_disc_number': track_info['disc_number'],
                    'track_duration_ms': track_info['duration_ms'],
                    'track_explicit': track_info['explicit'],
                    'track_href': track_info['href'],
                    'track_is_local': track_info['is_local'],
                    'track_name': track_info['name'],
                    'track_popularity': track_info['popularity'],
                    'track_preview_url': track_info['preview_url'],
                    'track_number': track_info['track_number'],
                    'track_uri': track_info['uri']
                })
            except:
                print(f'Error in trackload')
                traceback.print_exc()
                
        # 'track_external_ids': track_info['external_ids'],
        # 'track_external_urls': track_info['external_urls'],

        return tracks_data

    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()
        return None

sp, genius = connect_apis()
logging.basicConfig(filename='playlist_loader.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

for playlist_id, playlist in playlists.items():
    conn = get_pg_conn()
    cursor = conn.cursor()

    try:
        tracks_data = get_tracks(playlist_id)
        logging.info(f"{playlist['name']} - Playlist Load: Initiated.")
        for track in tracks_data:
            if track is not None:
                track_keys = ', '.join(track.keys())
                string_placeholder = ', '.join(['%s' for _ in track.keys()])
                track_values = [change_quotes(value) for value in track.values()]
                query = f"INSERT INTO track ({track_keys}) VALUES ({string_placeholder}) ON CONFLICT (track_id) DO NOTHING;"
                cursor.execute(query, track_values)
        conn.commit()
        logging.info(f"{playlist['name']} - Playlist Load: Completed.")
        pprint(f"{playlist['name']} - Playlist Load: Completed.")
        time.sleep(20)
    except Exception as e:
        pprint(f"{playlist['name']} - Playlist Load: Error.")
        logging.error(f"{playlist['name']} - Error during Playlist Load: {e}")
    finally:
        # Close database connections
        pprint(f"{playlist['name']} - Closing cursor.")
        cursor.close()

In [ ]:
def get_tracks(playlist_id):
    try:
        # Get playlist tracks using the playlist_tracks endpoint
        playlist_tracks = sp.playlist_tracks(playlist_id)

        # Extract relevant information from the track objects
        tracks_data = []

        for track in playlist_tracks['items']:
            try:
                track_info = track['track']
                tracks_data.append({
                    'track_id': track_info['id'],
                    'track_album': track_info['album']['name'],
                    'track_artists': f"{', '.join([artist['name'] for artist in track_info['artists']])}",
                    'track_available_markets': f"[{', '.join([available_markets for available_markets in track_info['available_markets']])}]",
                    'track_disc_number': track_info['disc_number'],
                    'track_duration_ms': track_info['duration_ms'],
                    'track_explicit': track_info['explicit'],
                    'track_href': track_info['href'],
                    'track_is_local': track_info['is_local'],
                    'track_name': track_info['name'],
                    'track_popularity': track_info['popularity'],
                    'track_preview_url': track_info['preview_url'],
                    'track_number': track_info['track_number'],
                    'track_uri': track_info['uri']
                })
            except:
                print(f'Error in trackload')
                traceback.print_exc()
                
        # 'track_external_ids': track_info['external_ids'],
        # 'track_external_urls': track_info['external_urls'],

        return tracks_data

    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()
        return None

sp, genius = connect_apis()
logging.basicConfig(filename='playlist_loader.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

for playlist_id, playlist in playlists.items():
    conn = get_pg_conn()
    cursor = conn.cursor()

    try:
        tracks_data = get_tracks(playlist_id)
        logging.info(f"{playlist['name']} - Playlist Load: Initiated.")
        for track in tracks_data:
            if track is not None:
                track_keys = ', '.join(track.keys())
                string_placeholder = ', '.join(['%s' for _ in track.keys()])
                track_values = [change_quotes(value) for value in track.values()]
                query = f"INSERT INTO track ({track_keys}) VALUES ({string_placeholder}) ON CONFLICT (track_id) DO NOTHING;"
                cursor.execute(query, track_values)
        conn.commit()
        logging.info(f"{playlist['name']} - Playlist Load: Completed.")
        pprint(f"{playlist['name']} - Playlist Load: Completed.")
        time.sleep(20)
    except Exception as e:
        pprint(f"{playlist['name']} - Playlist Load: Error.")
        logging.error(f"{playlist['name']} - Error during Playlist Load: {e}")
    finally:
        # Close database connections
        pprint(f"{playlist['name']} - Closing cursor.")
        cursor.close()

In [314]:
playlist_tracks_data = get_playlist_tracks(sp, '37i9dQZF1DXcBWIGoYBM5M')
if playlist_tracks_data:
    print("Playlist Tracks:")
    for track in playlist_tracks_data:
        pprint(", ".join(f"{key}={value}" for key, value in track.items()))
else:
    print("Failed to retrieve playlist tracks.")


conn = get_pg_conn()
cursor = conn.cursor()

for playlist_id, playlist in playlists.items():
    print(playlist)
    cursor.execute('''INSERT INTO playlist (playlist_id, playlist_name, playlist_description, playlist_url)
                    VALUES (%s, %s, %s, %s)''', (playlist['id'], playlist['name'], playlist['description'], playlist['href']))
conn.commit()

Playlist Tracks:
('id=4xhsWYTOGcal8zt0J161CU, name=Lovin On Me, artist=Jack Harlow, album=Lovin '
 'On Me, duration_ms=138411, popularity=92, '
 'preview_url=https://p.scdn.co/mp3-preview/eeeb6151d0049367b3ad3db0389e9e3b116c0774?cid=09adb02f86284fb9973f9bfdb29ce2e0, '
 'track_number=1, uri=spotify:track:4xhsWYTOGcal8zt0J161CU, is_local=False, '
 'href=https://api.spotify.com/v1/tracks/4xhsWYTOGcal8zt0J161CU, '
 'explicit=True, disc_number=1')
('id=3rUGC1vUpkDG9CZFHMur1t, name=greedy, artist=Tate McRae, album=greedy, '
 'duration_ms=131872, popularity=100, '
 'preview_url=https://p.scdn.co/mp3-preview/ca7bdcb691fb64a5af8a3253b89356390664fcf1?cid=09adb02f86284fb9973f9bfdb29ce2e0, '
 'track_number=1, uri=spotify:track:3rUGC1vUpkDG9CZFHMur1t, is_local=False, '
 'href=https://api.spotify.com/v1/tracks/3rUGC1vUpkDG9CZFHMur1t, '
 'explicit=True, disc_number=1')
('id=7dJYggqjKo71KI9sLzqCs8, name=Agora Hills, artist=Doja Cat, album=Scarlet, '
 'duration_ms=265360, popularity=92, '
 'preview_url

UniqueViolation: duplicate key value violates unique constraint "playlist_pkey"
DETAIL:  Key (playlist_id)=(37i9dQZF1DXcBWIGoYBM5M) already exists.


In [ ]:
conn = get_pg_conn()
cursor = conn.cursor()

for playlist_id, playlist in playlists.items():
    print(playlist)
    cursor.execute('''INSERT INTO playlist (playlist_id, playlist_name, playlist_description, playlist_url)
                    VALUES (%s, %s, %s, %s)''', (playlist['id'], playlist['name'], playlist['description'], playlist['href']))
conn.commit()

In [49]:
conn = get_pg_conn()
cursor = conn.cursor()

# Create Track Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS album (      
        album_id VARCHAR(255) PRIMARY KEY,
        album_name VARCHAR(255),
        album_type VARCHAR(255),
        album_release_date DATE,
        album_release_date_precision VARCHAR(25),
        album_total_tracks INTEGER,
        album_artists TEXT,
        album_href VARCHAR(255),
        album_uri VARCHAR(255)
    )
''')


# track_external_ids TEXT[],
# track_external_urls TEXT[],

# Commit changes and close the connection
conn.commit()

In [50]:
def get_albums(playlist_id):
    try:
        # Get playlist tracks using the playlist_tracks endpoint
        playlist_tracks = sp.playlist_tracks(playlist_id)

        # Extract unique album information from the track objects
        albums_data = {}

        for track in playlist_tracks['items']:
            track_info = track['track']
            album = track_info['album']
            
            # Check if album information is not already collected
            if album['id'] not in albums_data:
                album_data = {
                    'album_id': album['id'],
                    'album_name': album['name'],
                    'album_type': album['album_type'],
                    'album_release_date': album['release_date'],
                    'album_release_date_precision': album['release_date_precision'],
                    'album_total_tracks': album['total_tracks'],
                    'album_artists': [artist['name'] for artist in album['artists']],
                    'album_href': album['href'],
                    'album_uri': album['uri'],
                }

                albums_data[album['id']] = album_data

        return list(albums_data.values())

    except Exception as e:
        print(f"Error: {e}")
        return None
        
sp, genius = connect_apis()
logging.basicConfig(filename='album_loader.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


for playlist_id in playlists:
    conn = get_pg_conn()
    cursor = conn.cursor()

    try:
        album_data = get_albums(playlist_id)
        logging.info(f"{playlist['name']} - Album Load: Initiated.")
        for album in album_data:
            if album is not None:
                album_keys = ', '.join(album.keys())
                string_placeholder = ', '.join(['%s' for _ in album.keys()])
                album_values = [change_quotes(value) for value in album.values()]
                query = f"INSERT INTO album ({album_keys}) VALUES ({string_placeholder}) ON CONFLICT (album_id) DO NOTHING;"
                cursor.execute(query, album_values)
        conn.commit()
        logging.info(f"{playlist['name']} - Album Load: Completed.")
        pprint(f"{playlist['name']} - Album Load: Completed.")
        time.sleep(20)
    except Exception as e:
        traceback.print_exc()
        sys.exit()
        pprint(f"{playlist['name']} - Album Load: Error.")
        logging.error(f"{playlist['name']} - Error during Album Load: {e}")
    finally:
        # Close database connections
        pprint(f"{playlist['name']} - Closing cursor.")
        cursor.close()

{'album_id': '6VCO0fDBGbRW8mCEvV95af', 'album_name': 'Lovin On Me', 'album_type': 'single', 'album_release_date': '2023-11-10', 'album_release_date_precision': 'day', 'album_total_tracks': 1, 'album_artists': ['Jack Harlow'], 'album_href': 'https://api.spotify.com/v1/albums/6VCO0fDBGbRW8mCEvV95af', 'album_uri': 'spotify:album:6VCO0fDBGbRW8mCEvV95af'}
{'album_id': '3UOV8XvCwMKaATRNXrYCjN', 'album_name': 'greedy', 'album_type': 'single', 'album_release_date': '2023-09-15', 'album_release_date_precision': 'day', 'album_total_tracks': 1, 'album_artists': ['Tate McRae'], 'album_href': 'https://api.spotify.com/v1/albums/3UOV8XvCwMKaATRNXrYCjN', 'album_uri': 'spotify:album:3UOV8XvCwMKaATRNXrYCjN'}
{'album_id': '6DmPNcfpkXBVRJsEIJY9tl', 'album_name': 'Scarlet', 'album_type': 'album', 'album_release_date': '2023-09-22', 'album_release_date_precision': 'day', 'album_total_tracks': 17, 'album_artists': ['Doja Cat'], 'album_href': 'https://api.spotify.com/v1/albums/6DmPNcfpkXBVRJsEIJY9tl', 'album_

Traceback (most recent call last):
  File "C:\Users\sreek\AppData\Local\Temp\ipykernel_28928\160819914.py", line 53, in <module>
    cursor.execute(query, album_values)
psycopg2.errors.InvalidDatetimeFormat: invalid input syntax for type date: "1966-02"
LINE 1: ...g3z3Y', 'The Best Of The Animals', 'compilation', '1966-02',...
                                                             ^



AttributeError: 'tuple' object has no attribute 'tb_frame'

In [135]:
def get_artist_as_list(playlist_id):
    try:
        # Get playlist tracks using the playlist_tracks endpoint
        playlist_tracks = sp.playlist_tracks(playlist_id)

        # Extract unique artist information from the track objects
        artists_list = []

        for track in playlist_tracks['items']:
            track_info = track['track']

            for artist_info in track_info['artists']:
                artists_list.append(artist_info['id'])
        return artists_list

    except Exception as e:
        print(f"Error: {e}")
        return None

def get_all_artitst():
    all_artists = []
    for playlist_id in playlists.keys():
        artists = get_artist_as_list(playlist_id)
        if artists:
            for artist in artists:
                if artist is not None and artist not in all_artists:
                    all_artists.append(artist)

# all_artists = get_all_artitst()

In [8]:
def fetch_result_set(query):
    try:
        conn = get_pg_conn()
        
        cursor = conn.cursor()
        cursor.execute(query)

        # Fetch all track IDs
        track_ids = cursor.fetchall()

        # Close the cursor and connection
        cursor.close()
        conn.close()

        return track_ids

    except Exception as e:
        print(f"Error: {e}")
        return None

query = "SELECT track_id FROM track"
fetched_track_ids = fetch_result_set(query)
type(fetched_track_ids[0][0])

track_ids = [tracks[0] for tracks in fetched_track_ids]
len(track_ids)

54799

In [2]:
def get_audio_features_for_large_set(track_ids, sp):
    try:
        # Initialize an empty list to store all audio features
        all_audio_features = []

        # Split the track IDs into chunks of 100 (maximum allowed per request)
        track_id_chunks = [track_ids[i:i+100] for i in range(0, len(track_ids), 100)]

        # Fetch audio features for each chunk
        for chunk in track_id_chunks:
            audio_features_chunk = sp.audio_features(chunk)
            all_audio_features.extend(audio_features_chunk)
            print(all_audio_features[0].keys())
            print(all_audio_features[0]['id'])
            sys.exit()

        return all_audio_features

    except Exception as e:
        print(f"Error fetching audio features: {e}")
        return None

all_audio_features = get_audio_features_for_large_set(track_ids, sp)

In [160]:
def get_column_names(table_name):
    try:
        # Connect to the PostgreSQL database
        conn = get_pg_conn()
        cursor = conn.cursor()

        # Get the column names from the information schema
        query = f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}';"
        cursor.execute(query)

        # Fetch all column names
        column_names = [row[0] for row in cursor.fetchall()]

        # Close the cursor and connection
        cursor.close()
        conn.close()

        return column_names

    except Exception as e:
        print(f"Error: {e}")
        return None
        
get_column_names("track")

['track_is_local',
 'track_duration_ms',
 'track_popularity',
 'track_explicit',
 'track_number',
 'track_disc_number',
 'track_preview_url',
 'track_id',
 'track_uri',
 'track_album',
 'track_artists',
 'track_available_markets',
 'track_href',
 'track_name']

In [ ]:
SELECT * FROM "playlist",
SELECT count(*) FROM playlist;

SELECT * FROM "track",
SELECT count(*) FROM track;

SELECT * FROM "album",
SELECT count(*) FROM album;

SELECT * FROM "artist",
SELECT count(*) FROM artist;

SELECT * FROM "lyrics",
SELECT count(*) FROM lyrics;

SELECT * FROM "audio_features",
SELECT count(*) FROM audio_features;

-- SELECT * FROM user;

-- DROP TABLE lyrics;

In [170]:
# sp.audio_features(track_ids)

sp.tracks(track_ids[0:5])['tracks'][0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [164]:
sp.artists(all_artists[0:10])

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2LIk90788K0zvyj2JJVwkJ'},
   'followers': {'href': None, 'total': 3164524},
   'genres': ['deep underground hip hop',
    'kentucky hip hop',
    'pop rap',
    'rap'],
   'href': 'https://api.spotify.com/v1/artists/2LIk90788K0zvyj2JJVwkJ',
   'id': '2LIk90788K0zvyj2JJVwkJ',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5eb2aab40ce03f3fa86163f78bb',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab676161000051742aab40ce03f3fa86163f78bb',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f1782aab40ce03f3fa86163f78bb',
     'width': 160}],
   'name': 'Jack Harlow',
   'popularity': 82,
   'type': 'artist',
   'uri': 'spotify:artist:2LIk90788K0zvyj2JJVwkJ'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/45dkTj5sMRSjrmBSBeiHym'},
   'followers': {'href': None, 'total': 4149243},
   'genres': ['a

In [ ]:
def get_playlist_artists(playlist_id):
    try:
        # Get playlist tracks using the playlist_tracks endpoint
        playlist_tracks = sp.playlist_tracks(playlist_id)

        # Extract unique artist information from the track objects
        artists_data = {}

        for track in playlist_tracks['items']:
            track_info = track['track']

            for artist_info in track_info['artists']:
                artist_id = artist_info['id']

                # Check if artist information is not already collected
                if artist_id not in artists_data:
                    artist_data = {
                        'id': artist_id,
                        'name': artist_info['name'],
                        'href': artist_info['href'],
                        'uri': artist_info['uri'],
                        'type': artist_info['type'],
                    }

                    artists_data[artist_id] = artist_data

        return list(artists_data.values())

    except Exception as e:
        print(f"Error: {e}")
        return None

In [82]:
sp.playlist_tracks("37i9dQZF1DXcBWIGoYBM5M")['items'][0]['track']['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/2LIk90788K0zvyj2JJVwkJ'},
  'href': 'https://api.spotify.com/v1/artists/2LIk90788K0zvyj2JJVwkJ',
  'id': '2LIk90788K0zvyj2JJVwkJ',
  'name': 'Jack Harlow',
  'type': 'artist',
  'uri': 'spotify:artist:2LIk90788K0zvyj2JJVwkJ'}]

In [107]:
all_artists

['2LIk90788K0zvyj2JJVwkJ',
 '45dkTj5sMRSjrmBSBeiHym',
 '5cj0lLjcoR7YOSnhnX0Po5',
 '6M2wZ9GZgrQXHCFfjv46we',
 '7uMDnSZyUYNBPLhPMNuaM2',
 '06HL4z0CvFAxyc27GXpf02',
 '3SozjO3Lat463tQICI9LcE',
 '0Y5tJX1MQlPlqiwlOH1tJY',
 '7tYKF4w9nC0nq9CsPZTHyP',
 '6HaGTQPmzraVmaVxvz6EUc',
 '4q3ewBCX7sLwd24euuV69X',
 '5cj0lLjcoR7YOSnhnX0Po5',
 '45dkTj5sMRSjrmBSBeiHym',
 '1McMsnEElThX1knmY4oliG',
 '40ZNYROS4zLfyyBSs2PGe2',
 '70kkdajctXSbqSMJbQO424',
 '06HL4z0CvFAxyc27GXpf02',
 '2uYWxilOVlUdk4oV9DvwqK',
 '1Xyo4u8uXC1ZmMpatF05PJ',
 '699OTQXzgjhIYAHMy9RyPD',
 '6tbjWDEIzxoDsBA1FuhfPW',
 '0jbo7KFNMiIkfBR6ih0yhm',
 '6M2wZ9GZgrQXHCFfjv46we',
 '46pWGuE3dSwY3bMMXGBvVS',
 '0C8ZW7ezQVs4URX5aX7Kqx',
 '2RQXRUsr4IW1f3mKyKsy4B',
 '6HaGTQPmzraVmaVxvz6EUc',
 '3MdXrJWsbVzdn6fe5JYkSQ',
 '33qOK5uJ8AR2xuQQAhHump',
 '7tYKF4w9nC0nq9CsPZTHyP',
 '1McMsnEElThX1knmY4oliG',
 '6qqNVTkY8uBg9cP3Jd7DAH',
 '2hlmm7s2ICUX0LVIhVFlZQ',
 '246dkjvS1zLTtiykXe5h60',
 '2tIP7SsRs7vjIcLrU85W8J',
 '6HaGTQPmzraVmaVxvz6EUc',
 '5H4yInM5zmHqpKIoMNAx4r',
 

In [113]:
count = 0
for i in all_artists:
    if i is None:
        count+=1
count

0

In [ ]:
def get_playlist_artists(playlist_id):
    try:
        # Get playlist tracks using the playlist_tracks endpoint
        playlist_tracks = sp.playlist_tracks(playlist_id)

        # Extract unique artist information from the track objects
        artists_data = {}

        for track in playlist_tracks['items']:
            track_info = track['track']

            for artist_info in track_info['artists']:
                artist_id = artist_info['id']

                # Check if artist information is not already collected
                if artist_id not in artists_data:
                    artist_data = {
                        'id': artist_id,
                        'name': artist_info['name'],
                        'href': artist_info['href'],
                        'uri': artist_info['uri'],
                        'type': artist_info['type'],
                    }

                    artists_data[artist_id] = artist_data

        return list(artists_data.values())

    except Exception as e:
        print(f"Error: {e}")
        return None
        
sp, genius = connect_apis()
logging.basicConfig(filename='album_loader.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


for playlist_id, playlist in playlists.items():
    conn = get_pg_conn()
    cursor = conn.cursor()

    try:
        album_data = get_albums(playlist_id)
        logging.info(f"{playlist['name']} - Album Load: Initiated.")
        for album in album_data:
            if album is not None:
                album_keys = ', '.join(album.keys())
                string_placeholder = ', '.join(['%s' for _ in album.keys()])
                album_values = [change_quotes(value) for value in album.values()]
                query = f"INSERT INTO album ({album_keys}) VALUES ({string_placeholder}) ON CONFLICT (album_id) DO NOTHING;"
                cursor.execute(query, album_values)
        conn.commit()
        logging.info(f"{playlist['name']} - Album Load: Completed.")
        pprint(f"{playlist['name']} - Album Load: Completed.")
        time.sleep(20)
    except Exception as e:
        traceback.print_exc()
        sys.exit()
        pprint(f"{playlist['name']} - Album Load: Error.")
        logging.error(f"{playlist['name']} - Error during Album Load: {e}")
    finally:
        # Close database connections
        pprint(f"{playlist['name']} - Closing cursor.")
        cursor.close()

In [211]:
conn = get_pg_conn()
cursor = conn.cursor()

# Create Track Table
cursor.execute('''
    CREATE TABLE artist (
        artist_id VARCHAR(255) PRIMARY KEY,
        artist_name VARCHAR(255) NOT NULL,
        followers INT,
        genres TEXT,
        popularity INT
    )
''')

# Commit changes and close the connection
conn.commit()

In [213]:
def get_artist_details(sp, artist_ids):

    all_genres = []
    # Set the number of artists to request per API call
    batch_size = 50

    # Paginate through the artist IDs
    for offset in range(17100, len(artist_ids), batch_size):
        # Extract a batch of artist IDs
        artist_ids_batch = artist_ids[offset:offset + batch_size]

        # Fetch artist details for the current batch
        artists = sp.artists(artist_ids_batch)
        genres = process_artists(artists)

        for genre in genres:
            if genre not in all_genres:
                all_genres.append(genre)
        
        remaining_artist_ids = len(artist_ids) - (offset + batch_size)
        processed_artist_ids = offset + len(artist_ids_batch)
        print(f"Remaining artist IDs: {remaining_artist_ids}\nProgress: {processed_artist_ids} / {len(artist_ids)}", end='\r')

        # Introduce a delay to avoid rate limiting (adjust as needed)
        time.sleep(10)
    return all_genres
        

def process_artists(artists):
    # Connect to PostgreSQL
    try:
        all_genres = []
        
        # Open a cursor to perform database operations
        conn = get_pg_conn()
        cursor = conn.cursor()

        # Process each artist and store in the PostgreSQL table
        for artist in artists['artists']:
            for genre in artist['genres']:
                if genre not in all_genres:
                    all_genres.append(genre)
            
            artist_id = artist['id']
            artist_name = artist['name'].replace("'", "''")
            followers = artist['followers']['total']
            genre_str = ', '.join([genre.replace("'", "''") for genre in artist['genres']])
            genres = f"[{genre_str}]"
            popularity = artist['popularity']
            
            # Replace 'your_table_name' with your actual table name
            query = f"INSERT INTO artist (artist_id, artist_name, followers, genres, popularity) " \
                    f"VALUES ('{artist_id}', '{artist_name}', {followers}, '{genres}', {popularity}) ON CONFLICT (artist_id) DO NOTHING;"

            cursor.execute(query)

        # Commit the transaction
        conn.commit()
        cursor.close()
        conn.close()
        return all_genres

    except Exception as e:
        print(f"Error processing artists: {e}")

sp, genius = connect_apis()
all_genres = get_artist_details(sp, all_artists)
pprint(len(all_genres))
all_genres

Remaining artist IDs: 13920
Remaining artist IDs: 13870
Remaining artist IDs: 13820
Remaining artist IDs: 13770
Remaining artist IDs: 13720
Remaining artist IDs: 13670
Remaining artist IDs: 13620
Remaining artist IDs: 13570
Remaining artist IDs: 13520
Remaining artist IDs: 13470
Remaining artist IDs: 13420
Remaining artist IDs: 13370
Remaining artist IDs: 13320
Remaining artist IDs: 13270
Remaining artist IDs: 13220
Remaining artist IDs: 13170
Remaining artist IDs: 13120
Remaining artist IDs: 13070
Remaining artist IDs: 13020
Remaining artist IDs: 12970
Remaining artist IDs: 12920
Remaining artist IDs: 12870
Remaining artist IDs: 12820
Remaining artist IDs: 12770
Remaining artist IDs: 12720
Remaining artist IDs: 12670
Remaining artist IDs: 12620
Remaining artist IDs: 12570
Remaining artist IDs: 12520
Remaining artist IDs: 12470
Remaining artist IDs: 12420
Remaining artist IDs: 12370
Remaining artist IDs: 12320
Remaining artist IDs: 12270
Remaining artist IDs: 12220
Remaining artist IDs

['banda',
 'grupera',
 'gruperas inmortales',
 'grupero romantico',
 'musica mexicana',
 'norteno',
 'tejano',
 'mexican son',
 'son jarocho',
 'mariachi',
 'african reggae',
 'sierra leonean pop',
 'world',
 'folklore peruano',
 'musica afroperuana',
 'musica criolla',
 'cantautor',
 'folclore tucumano',
 'folklore argentino',
 'nueva cancion',
 'trova',
 'afropop',
 'south african choral',
 'south african jazz',
 'arab folk',
 'arabic jazz',
 'oud',
 'ecm-style jazz',
 'bossa nova',
 'mpb',
 'musica popular mineira',
 'samba',
 'brazilian jazz',
 'jazz guitar',
 'samba-jazz',
 'velha guarda',
 'violao',
 'desert blues',
 'malian blues',
 'mande pop',
 'wassoulou',
 'chicha',
 'folclor afrocolombiano',
 'latin afrobeat',
 'nu-cumbia',
 'polish folk',
 'belgian hip hop',
 'zim urban groove',
 'ethio-jazz',
 'jazz flute',
 'jazz funk',
 'soul jazz',
 'new romantic',
 'new wave pop',
 'solo wave',
 'sophisti-pop',
 'classic soundtrack',
 'vintage italian soundtrack',
 'british jazz',
 'j

In [ ]:
def get_song_lyrics(song_name, artist_name):
    try:
        # Get the first track found on Spotify
        sp, genius = connect_apis()

        # Search for the song on Genius using the artist and title
        genius_results = genius.search_song(song_name, artist_name)
        if genius_results:
            # Print and return the lyrics
            print(f"Lyrics for '{song_name}' by {artist_name}:\n")
            #print(genius_results.lyrics)
            return genius_results.lyrics
        else:
            print(f"Lyrics for '{song_name}' not found on Genius.")
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None



In [236]:
len(all_lyrics)

9666

In [245]:
conn = get_pg_conn()
cursor = conn.cursor()

# Create Track Table
cursor.execute('''
    CREATE TABLE lyrics (
        track_id VARCHAR(255) PRIMARY KEY,
        track_lyrics TEXT
    )
''')

# Commit changes and close the connection
conn.commit()

In [ ]:
query = "SELECT track_id, track_name, SPLIT_PART(track_artists, ',', 1) FROM track"
fetch_track_name_artist = fetch_result_set(query)
track_name_artist = [(item[0], item[1], item[2]) for item in fetch_track_name_artist]

conn = get_pg_conn()
cursor = conn.cursor()

all_lyrics = dict()
for track_id, track_name, track_artist in track_name_artist:
    try:
        logging.info(f"{track_id} - Track Lyrics Load: Initiated.")
        track_lyrics = get_song_lyrics(track_name, track_artist)
        all_lyrics[track_id] = track_lyrics
        query = f"INSERT INTO lyrics (track_id, track_lyrics) VALUES (%s, %s) ON CONFLICT (track_id) DO NOTHING;"
        cursor.execute(query, (track_id, track_lyrics))
        logging.info(f"{track_id} - Track Lyrics Load: Completed.")
        pprint(f"{track_id} - Playlist Load: Completed.")
    except:
        logging.info(f"{track_id} - Track Lyrics Load: Error.")
        pprint(f"{track_id} - Track Lyrics Load: Error.")
        trackback.print_exc()

In [246]:
conn = get_pg_conn()
cursor = conn.cursor()
## insert into the lyrics table
try:
    for track_id, track_lyrics in all_lyrics.items():
        logging.info(f"{track_id} - Track Lyrics Load: Initiated.")
        query = f"INSERT INTO lyrics (track_id, track_lyrics) VALUES (%s, %s) ON CONFLICT (track_id) DO NOTHING;"
        cursor.execute(query, (track_id, track_lyrics))
        logging.info(f"{track_id} - Track Lyrics Load: Completed.")
        pprint(f"{track_id} - Playlist Load: Completed.")
    conn.commit()
except Exception as e:
    traceback.print_exc()
    logging.info(f"{track_id} - Track Lyrics Load: Error.")
    pprint(f"{track_id} - Track Lyrics Load: Error.")
finally:
    # Close database connections
    logging.info(f"{track_id} - Track Lyrics - Connection closing")
    cursor.close()

# conn = get_pg_conn()
# cursor = conn.cursor()

# for playlist_id, playlist in playlists.items():
#     print(playlist)
#     cursor.execute('''INSERT INTO playlist (playlist_id, playlist_name, playlist_description, playlist_url)
#                     VALUES (%s, %s, %s, %s)''', (playlist['id'], playlist['name'], playlist['description'], playlist['href']))

'0UWhRKpZxLEUZhmku98WcH - Playlist Load: Completed.'
'5yEpB2g3CbuVk4YmAkzkOK - Playlist Load: Completed.'
'61SdG3F3LncuuMLAS8oN6j - Playlist Load: Completed.'
'61ydRWnQ31DWFDqEj325nf - Playlist Load: Completed.'
'2AX3BnBmNwR2lHZhkKvmXJ - Playlist Load: Completed.'
'1OGCx6z4xbV7oPryuLx8mP - Playlist Load: Completed.'
'2AFJkvRUFVx0MccykGaHV2 - Playlist Load: Completed.'
'2tKvPt023QVMNkkNqmTDap - Playlist Load: Completed.'
'0uUy3WK0JtTqhSFikqprgz - Playlist Load: Completed.'
'0ge66NW3sKKgWW7eKQHnE0 - Playlist Load: Completed.'
'18N4jiKRHtC5v5bgMtFeUF - Playlist Load: Completed.'
'6QUppE7S4HgcfC3jZIcYlV - Playlist Load: Completed.'
'0jVm8tMbpsDN7goNU6NyzL - Playlist Load: Completed.'
'0qyQKbbPLaG9FIENFTL7KY - Playlist Load: Completed.'
'6I9mM7Q3YXKYSppopUuCQN - Playlist Load: Completed.'
'1QN5cKoYlEkJ4aMBfqDqXe - Playlist Load: Completed.'
'7By7UYWA21PcKoKCl8ZaI7 - Playlist Load: Completed.'
'53DoKGKHCae8SekXxqUqys - Playlist Load: Completed.'
'6Alshe3WYMVZRbYWgijYWT - Playlist Load: Compl

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_spotify_listening_history(sp, limit=50):
    # Get the current user's username
    username = sp.me()['id']

    # Get the user's listening history
    results = sp.current_user_recently_played(limit=limit)

    # Extract relevant information from the results
    listening_history = []

    for item in results['items']:
        track_info = item['track']
        artist_info = track_info['artists'][0]
        
        listening_history.append({
            'track_name': track_info['name'],
            'artist_name': artist_info['name'],
            'played_at': item['played_at'],
            'album_name': track_info['album']['name']
        })

    return listening_history


# Get the user's listening history (adjust the limit as needed)
listening_history = get_spotify_listening_history(sp, limit=50)

# Print the listening history
for i, track in enumerate(listening_history, 1):
    print(f"{i}. {track['track_name']} by {track['artist_name']} from {track['album_name']} (played at {track['played_at']})")


In [309]:
limit = 50
# 2023-11-30T01:34:14.506Z
sp.current_user_recently_played(limit=limit, before='2023-11-30T01:34:14.506Z')['items'][0]['played_at']

'2023-11-30T01:22:57.931Z'

In [319]:
sp.me().keys()
# len(sp.current_user_playlists()['items'])
# pd.DataFrame(sp.user_playlists(sp.me()['id'])['items'])

len(sp.user_playlists(sp.me()['id']))

7

In [324]:
def get_user_id(username, sp):
    try:
        # Retrieve user details using the Spotify Web API
        user_details = sp.user(username)

        # Extract user ID
        user_id = user_details['id']
        return user_id

    except Exception as e:
        print(f"Error fetching user ID: {e}")
        return None
        
get_user_id("WeeBeeS", sp)

'weebees'

In [337]:
def get_user_details_by_id(user_id, sp):
    try:
        # Retrieve user details using the Spotify Web API
        user_details = sp.user(user_id)
        
        # Extract relevant information
        details = {
            'display_name': user_details.get('display_name'),
            'id': user_details.get('id'),
            'email': user_details.get('email'),
            'followers': user_details.get('followers', {}).get('total'),
            'country': user_details.get('country'),
        }

        # Get the user's recently played tracks
        recent_tracks = sp.user_playlists(user_details, limit=50)  # Adjust the limit as needed

        # Extract relevant information from recent tracks
        tracks = [{
            'track_name': track['track']['name'],
            'artist_name': track['track']['artists'][0]['name'],
            'played_at': track['played_at'],
            'album_name': track['track']['album']['name'],
        } for track in recent_tracks['items']]

        details['recent_tracks'] = tracks

        return details

    except Exception as e:
        print(f"Error fetching user details: {e}")
        traceback.print_exc()
        return None

def print_user_playlists(user_id, sp):
    try:
        # Initialize an empty list to store all playlists
        all_playlists = []

        # Initial request to get the first batch of playlists
        playlists = sp.user_playlists(user_id)

        # Append the playlists to the list
        all_playlists.extend(playlists['items'])

        # Check for more playlists and continue fetching until none are left
        while playlists['next']:
            playlists = sp.next(playlists)
            all_playlists.extend(playlists['items'])

        # Print the public playlists
        if all_playlists:
            print(f"Public Playlists for user {user_id}:")
            for i, playlist in enumerate(all_playlists, 1):
                print(f"{i}. {playlist['name']} (ID: {playlist['id']})")
        else:
            print(f"No public playlists found for user {user_id}")

    except Exception as e:
        print(f"Error fetching user playlists: {e}")

for user_id in ["ygk025r29ofb0yexgz9jkfxdo", 
                "313jpb7nsygrjtu7owt42lsug3dm", 
                "ygk025r29ofb0yexgz9jkfxdo", 
                "sanchibhaley", 
                "313v3mo6bvh5cihtfmz6pvkm3rhy", 
                "1221129264",
                "31ecztytamjcxxdoy2ciqxo2yqxi",
                "chilledcow",
               ]:
    print(print_user_playlists(user_id, sp))
    # print(user_id)

Error fetching user playlists: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
None
Error fetching user playlists: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
None
Public Playlists for user ygk025r29ofb0yexgz9jkfxdo:
1. sick transitions  (ID: 4SSEZktmb8rdmLabzXEkQj)
2. Homer Radio : EP. 1 - EP. 52 (ID: 2Jn13Pp8XkL0qHPrrVGoAE)
3. SAMPLES (ID: 1PrGADl6U3bidN7oe7zUUN)
4. My life is a movie (ID: 0k6xcLMhHiC0PHiLIbj49I)
5. Wait for the beat switch! (ID: 3BXIlSUx2InRS1HqE2rpRJ)
6. Vintage classics (ID: 30xObbdYZad7GPs87FefUp)
7. Midnight blues (ID: 3zY2nsR9AW734ISG3mjRor)
8. Rap (ID: 4Kv1RmL4vkAAd0Nw3gCmnb)
9. Desi tunes for the blues (ID: 5q5NTMEA6lptHlAuCTrnAd)
None
Public Playlists for user sanchibhaley:
1. Bolly Shit I grew up with (ID: 5sV7uSXIeBnobbU8BhdS5e)
2. Workout Sesh (ID: 2knyyhuqmpv3aoNw7WGbAx)
3. Genius (ID: 3kkgIgqQpihXIWl02E9s6A)
4. Scores (ID: 3EFfAAFXLnn1iic7BZo67Q)
5. Punjabi Tad

In [ ]:
def get_user_id(username, sp):
    try:
        # Search for playlists owned by the specified username
        results = sp.search(q=f'user:{username}', type='playlist')

        # Extract the user ID from the first playlist (assuming it exists)
        if 'playlists' in results and 'items' in results['playlists']:
            first_playlist = results['playlists']['items'][0]
            owner_info = first_playlist.get('owner', {})
            user_id = owner_info.get('id')

            return user_id

    except Exception as e:
        print(f"Error fetching user ID: {e}")

    return None

req_ids = ["ygk025r29ofb0yexgz9jkfxdo", "313jpb7nsygrjtu7owt42lsug3dm", "ygk025r29ofb0yexgz9jkfxdo", "sanchibhaley", "313v3mo6bvh5cihtfmz6pvkm3rhy", "1221129264",]

user_df = pd.read_csv("input_users.csv")

for user in user_df['username']:
    print(user)
    # get_user_id("weebees", sp)

In [ ]:
def get_user_id(username, sp):
    try:
        # Search for playlists owned by the specified username
        results = sp.search(q=f'user:{username}', type='playlist')

        # Extract the user ID from the first playlist (assuming it exists)
        if 'playlists' in results and 'items' in results['playlists']:
            first_playlist = results['playlists']['items'][0]
            owner_info = first_playlist.get('owner', {})
            user_id = owner_info.get('id')

            return user_id

    except Exception as e:
        print(f"Error fetching user ID: {e}")

    return None

get_user_id("weebees", sp)

In [251]:
sp, genius = connect_apis()
print(get_song_lyrics("Ten", "Fred Again"))

Searching for "Ten" by Fred Again...
Done.
Lyrics for 'Ten' by Fred Again:

9 Contributors​ten Lyrics[Intro]
(Back it up and dump it, back it up and dump it)
(Back it up and dump it, back it up and dump it)
Ten
(I can't lift it up, so good, can't get enough)
(Too thick, can't lift it up, I got that)
Ten (Ten, ten, ten)
Ten days, ten days, ten days

[Drop]
(Back it up and dump it, back it up and dump it)
Ten (Back it up and dump it, back it up and dump it)
Ten days, ten days, ten days (Ten, ten, ten)
(I can't lift it up, so good, can't get enough)
Ten (Too thick, can't lift it up, I got that dump truck)
(Too thick, can't)

[Bridge: Jozzy]
Though you know I don't speak too much
Walk like I don't speak too much
You know I'm comin'
Got me ridin' round with you
Got me ridin' round with you
You know I'm comin'
Got me ridin' round with you
Got me ridin' round with you
You know I'm
You might also like[Refrain: Jim Legxacy]
Yeah, ten days in a foreign ends without you here (Yeah, yeah, uh, uh, 

In [ ]:
conn.close()

In [5]:
# Example: Get the current user's saved tracks
results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(f"{idx + 1}: {track['name']} by {', '.join([artist['name'] for artist in track['artists']])}")

# Example: Search for a track
search_query = "Bohemian Rhapsody"
search_results = sp.search(q=search_query, type='track', limit=10)

print("Search Results for", search_query)
for idx, track in enumerate(search_results['tracks']['items']):
    print(f"{idx + 1}: {track['name']} by {', '.join([artist['name'] for artist in track['artists']])}")

1: Where You Want by Riton, David Guetta, Jozzy
2: Talk About by Rain Radio, DJ Craig Gorman
3: So U Kno by Overmono
4: I See Colors by Disco Lines, Rain Radio
5: 365 by Biscits, Camden Cox
6: Forbidden Feelingz by Nia Archives
7: Food for the Soul by it's murph
8: On My Knees by RÜFÜS DU SOL
9: San Frandisco by Dom Dolla
10: Palm Beach Banga by FISHER
11: Waves & Wavs by Ahmed Spins, Lizwi
12: Wanna Go Dancin' by FISHER
13: In The Yuma (feat. Aatig) by Chris Lake, Aatig
14: Yeah The Girls by FISHER, MERYLL
15: It's A Killa by FISHER, Shermanology
16: Drop The Game by Flume, Chet Faker
17: No Diggity by Chet Faker
18: ten by Fred again.., Jozzy
19: Sweet Lies by ESSEL
20: Price by PALASTIC
Search Results for Bohemian Rhapsody
1: Bohemian Rhapsody - Remastered 2011 by Queen
2: Bohemian Rhapsody by Queen
3: Bohemian Rhapsody by Panic! At The Disco
4: Bohemian Rhapsody - Live Aid by Queen
5: Bohemian Rhapsody - Remastered by Queen
6: Bohemian Rhapsody - Remastered 2011 by Queen
7: Bohemia

In [6]:
def get_user_playlists(sp, username):
    # Get the user's playlists
    playlists = []
    printed = False
    offset = 0
    limit = 20  # Maximum number of playlists per request
    while True:
        results = sp.user_playlists(username, offset=offset, limit=limit)
        playlists.extend(results['items'])
        offset += len(results['items'])
        if len(results['items']) < limit:
            break

    # Store playlist names and IDs in a list of dictionaries
    user_playlists = []
    for playlist in playlists:
        if not printed:
            print(playlist)
            print(playlist.keys())
            printed = True
        user_playlists.append({
            'name': playlist['name'],
            'id': playlist['id']
        })
    return user_playlists

In [7]:
all_spotify_playlists = get_user_playlists(sp, "spotify")

{'collaborative': False, 'description': 'Drake on top of the Hottest 50!', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DXcBWIGoYBM5M'}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXcBWIGoYBM5M', 'id': '37i9dQZF1DXcBWIGoYBM5M', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706f00000003fc26ce35d4ccd387685e041d', 'width': None}], 'name': "Today's Top Hits ", 'owner': {'display_name': 'Spotify', 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'}, 'href': 'https://api.spotify.com/v1/users/spotify', 'id': 'spotify', 'type': 'user', 'uri': 'spotify:user:spotify'}, 'primary_color': None, 'public': True, 'snapshot_id': 'MTY5NzkwNzYwMCwwMDAwMDAwMGZlYjhiNzYzYjI1YjI5MDE0YWY4ZTdjZDljNGM0OGIw', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXcBWIGoYBM5M/tracks', 'total': 50}, 'type': 'playlist', 'uri': 'spotify:playlist:37i9dQZF1DXcBWIGoYBM5M'}
dict_keys(['collaborative', 'description', 'external_urls',

In [8]:
all_spotify_playlists

[{'name': "Today's Top Hits ", 'id': '37i9dQZF1DXcBWIGoYBM5M'},
 {'name': 'RapCaviar', 'id': '37i9dQZF1DX0XUsuxWHRQd'},
 {'name': 'Hot Country', 'id': '37i9dQZF1DX1lVhptIYRda'},
 {'name': 'Viva Latino', 'id': '37i9dQZF1DX10zKzsJ2jva'},
 {'name': 'New Music Friday', 'id': '37i9dQZF1DX4JAvHpjipBk'},
 {'name': 'Peaceful Piano', 'id': '37i9dQZF1DX4sWSpwq3LiO'},
 {'name': 'RNB X', 'id': '37i9dQZF1DX4SBhb3fqCJd'},
 {'name': 'Rock Classics', 'id': '37i9dQZF1DWXRqgorJj26U'},
 {'name': 'mint', 'id': '37i9dQZF1DX4dyzvuaRJ0n'},
 {'name': 'Rock This', 'id': '37i9dQZF1DXcF6B6QPhFDv'},
 {'name': 'New Music Friday', 'id': '37i9dQZF1DWXJfnUiYjUKT'},
 {'name': 'just hits', 'id': '37i9dQZF1DXcRXFNfZr7Tp'},
 {'name': 'All Out 2000s', 'id': '37i9dQZF1DX4o1oenSJRJd'},
 {'name': 'All Out 90s', 'id': '37i9dQZF1DXbTxeAdrVG2l'},
 {'name': 'All Out 80s', 'id': '37i9dQZF1DX4UtSsGT1Sbe'},
 {'name': 'All Out 70s', 'id': '37i9dQZF1DWTJ7xPn4vNaz'},
 {'name': 'All Out 60s', 'id': '37i9dQZF1DXaKIA8E7WcJj'},
 {'name': 

In [9]:
len(all_spotify_playlists)

1108

In [10]:
save_this_song = None

def clean_lyrics(lyrics, song_title):
    # Find the position of the first occurrence of the song title
    title_position = lyrics.find(song_title)

    # If the song title is found, keep only the text after it
    if title_position != -1:
        cleaned_lyrics = lyrics[title_position + len(song_title):]
    else:
        # If the song title is not found, return the original lyrics
        cleaned_lyrics = lyrics

    return cleaned_lyrics
    
def get_playlist_audio_features(sp, playlist_id):
    playlist = sp.playlist_tracks(playlist_id)
    playlist_tracks = playlist['items']

    # Initialize lists to store the attributes
    track_names = []
    artist_names = []
    track_uris = []
    durations_ms = []
    popularity = []

    audio_features = []
    lyrics = []
    printed_already = False

    for track_info in playlist_tracks:
        track = track_info['track']
        track_names.append(track['name'])
        artist_names.append(', '.join([artist['name'] for artist in track['artists']]))
        track_uris.append(track['uri'])
        durations_ms.append(track['duration_ms'])
        popularity.append(track['popularity'])

        # Get audio features for the track
        # audio = sp.audio_features(track['uri'])
        audio = False
        if audio:
            audio_features.append(audio[0])
        else:
            audio_features.append(None)

        # # Search for lyrics on Genius
        # try:
        #     song = genius.search_song(track['name'], artist_names[-1])
        #     if not printed_already:
        #         printed_already = True
        #         save_this_song = song
        #         print(song)
        #     if song:
        #         lyrics.append(clean_lyrics(song.lyrics, song.title))
        #     else:
        #         lyrics.append(None)
        # except:
        #     lyrics.append(None)
        time.sleep(1)

    # Create a DataFrame
    df = pd.DataFrame({
        'track_name': track_names,
        'artist_name': artist_names,
        'track_uri': track_uris,
        'duration': durations_ms,
        'popularity': popularity,
        # 'lyrics': lyrics
    })

    # Add audio features to the DataFrame
    # df_audio = pd.DataFrame(audio_features)
    # df = pd.concat([df, df_audio], axis=1)

    return df

In [ ]:
if not os.path.exists("Data/"):
    os.makedirs("Data/")
    
for playlist in all_spotify_playlists:
    playlist_id = playlist['id']
    playlist_name = playlist['name']
    if playlist_name is not None:
        try:
            playlist_df = get_playlist_audio_features(sp, playlist_id)
            
            playlist_df.to_csv(f"Data/{playlist_name}.csv", index=False)
            print(f"{playlist_name} csv generated.")
        except:
            print(f"{playlist_name} failed.")
            traceback.print_exc()
        time.sleep(10)

    # if not os.path.exists(f"{playlist_name}.csv"):
    #     try:
    #         playlist_df = get_playlist_audio_features(sp, playlist_id)
            
    #         playlist_df.to_csv(f"Data/{playlist_name}.csv", index=False)
    #         print(f"{playlist_name} csv generated.")
    #     except:
    #         print(f"{playlist_name} failed.")
    #         traceback.print_exc()
    #     time.sleep(120)
    # else:
    #     print(f"{playlist_name} already exists.")
        
playlist_id = all_spotify_playlists[0]['id']
playlist_df = get_playlist_audio_features(sp, playlist_id)
print(playlist_df)